In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nfl_data_py as nfl
import ssl
ssl._create_default_https_context = ssl._create_stdlib_context
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
rolling_game_window = 4
rolling_series_window = 3
rolling_play_window = 30
rolling_play_quarter_window = 20

## nfl-data-py

In [3]:
# 1999-2021 available for analysis. Some properties are not available in earlier years.  Offensive formation available starting in 2022
start_season = 2022
end_season = 2023

end_season = end_season + 1
seasons = [*range(start_season, end_season, 1)]

draft_years = [*range(2000, 2024, 1)]

print(seasons)
print(draft_years)

[2022, 2023]
[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


## DATA DICTIONARY

In [4]:
# Display https://nflreadr.nflverse.com/articles/dictionary_pbp.html
# from IPython.display import IFrame
# IFrame(src='https://nflreadr.nflverse.com/articles/dictionary_pbp.html', width=1000, height=600)

## NFL PLAY-BY-PLAY DATA

In [5]:
play_data = nfl.import_pbp_data(years=seasons, downcast=False, cache=False) # , alt_path=None)

# Clean dataframe using nfl-data-api cleaning function
play_df = nfl.clean_nfl_data(play_data)

play_df.shape

2022 done.
2023 done.


(97546, 390)

In [6]:
# Print columns
print(play_df.columns.tolist())

['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team', 'td_player_name', 'td_player_id', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential

In [7]:
# Preliminary play_type filter 'None' and 'no play'
play_df = play_df.loc[~(play_df['play_type'].isin([None,'no_play']))]

play_df.shape

(85064, 390)

In [8]:
# Modify team names to match other data sources
play_df['home_team'] = np.where(play_df['home_team'] == 'OAK', 'LV', play_df['home_team'])
play_df['away_team'] = np.where(play_df['away_team'] == 'OAK', 'LV', play_df['away_team'])
play_df['posteam'] = np.where(play_df['posteam'] == 'OAK', 'LV', play_df['posteam'])
play_df['defteam'] = np.where(play_df['defteam'] == 'OAK', 'LV', play_df['defteam'])
play_df['penalty_team'] = np.where(play_df['penalty_team'] == 'OAK', 'LV', play_df['penalty_team'])
play_df['side_of_field'] = np.where(play_df['side_of_field'] == 'OAK', 'LV', play_df['side_of_field'])
play_df['game_id'] = play_df['game_id'].str.replace('OAK', 'LV', case=True)

# Convert game_date to datetime format
play_df['game_date']= pd.to_datetime(play_df['game_date'])

# Create year column from game_date using isoformat
play_df['year'] = play_df['game_date'].dt.year
play_df['month'] = play_df['game_date'].dt.month
play_df['day'] = play_df['game_date'].dt.day

play_df['game_alt_id'] = play_df['home_team'] + '_' + play_df['away_team'] + '_' +  play_df['year'].astype(str) + '_' + play_df['month'].astype(str).str.zfill(2) + '_' + play_df['day'].astype(str).str.zfill(2)
    
play_df.drop(['year','month','day'], axis=1, inplace=True)

play_df.sample(1)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type,game_alt_id
42461,3688.0,2022_16_WAS_SF,2022122408,SF,WAS,REG,16,SF,home,WAS,...,00-0035647;00-0034333;00-0036614;00-0033084;00...,11.0,11.0,NaN,NaN,NaN,None,None,None,SF_WAS_2022_12_24


In [9]:
# Move game_alt_id to fourth column in dataframe
cols = play_df.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
play_df = play_df[cols]

In [10]:
play_df.home_team.nunique()

32

In [11]:
# Game-play sequence
play_df['play'] = play_df.groupby(['game_id'])['play_id'].cumcount() + 1

play_df.play.nunique()

208

## NFL GAME SCHEDULE DATA

In [12]:
game_data = nfl.import_schedules(years=seasons)

# Clean dataframe using nfl-data-api cleaning function
game_df = nfl.clean_nfl_data(game_data)

# # Sort dataframe
game_df = game_df.sort_values(by=['season','week','gameday'], ascending=True)

game_df.shape

(562, 46)

In [13]:
game_df.head(2)

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6137,2022_01_BUF_LA,2022,REG,1,2022-09-08,Thursday,20:20,BUF,31.0,LA,...,NaN,00-0034857,00-0026498,Josh Allen,Matthew Stafford,Sean McDermott,Sean McVay,Carl Cheffers,LAX01,SoFi Stadium
6138,2022_01_NO_ATL,2022,REG,1,2022-09-11,Sunday,13:00,NO,27.0,ATL,...,NaN,00-0031503,00-0032268,Jameis Winston,Marcus Mariota,Dennis Allen,Arthur Smith,Alex Kemp,ATL97,Mercedes-Benz Stadium


In [14]:
# Modify team codes to match schedule
game_df['home_team'] = np.where(game_df['home_team'] == 'OAK', 'LV', game_df['home_team'])
game_df['away_team'] = np.where(game_df['away_team'] == 'OAK', 'LV', game_df['away_team'])
game_df['game_id'] = game_df['game_id'].str.replace('OAK', 'LV', case=True)

In [15]:
game_df.home_team.nunique()

32

In [16]:
game_df.away_team.nunique()

32

In [17]:
# Encode year month and day from gameday using isoformat
game_df['gameday'] = pd.to_datetime(game_df['gameday'])

game_df['year'] = game_df['gameday'].dt.year
game_df['month'] = game_df['gameday'].dt.month
game_df['day'] = game_df['gameday'].dt.day

game_df['game_alt_id'] = game_df['home_team'] + '_' + game_df['away_team'] + '_' + game_df['year'].astype(str) + '_' + game_df['month'].astype(str).str.zfill(2) + '_' + game_df['day'].astype(str).str.zfill(2)
    
game_df.drop(['year','month','day'], axis=1, inplace=True)

# Create game_id lookup table (joined with QBR table)
game_df = game_df[['game_id','game_alt_id','gameday','weekday','gametime','under_odds','over_odds','away_rest','home_rest']]

game_df.sample(2)

,game_id,game_alt_id,gameday,weekday,gametime,under_odds,over_odds,away_rest,home_rest
6585,2023_12_GB_DET,DET_GB_2023_11_23,2023-11-23,Thursday,12:30,-108.0,-112.0,4,4
6671,2023_17_NO_TB,TB_NO_2023_12_31,2023-12-31,Sunday,13:00,-112.0,-108.0,10,7


## NFL QBR DATA

In [18]:
qbr_data = nfl.import_qbr(years=seasons, level='nfl', frequency='weekly')

# Clean dataframe using nfl-data-api cleaning function
qbr_df = nfl.clean_nfl_data(qbr_data)

qbr_df.shape

(1108, 30)

In [19]:
qbr_df.sample(2)

,season,season_type,game_id,game_week,week_text,team_abb,player_id,name_short,rank,qbr_total,...,name_last,name_display,headshot_href,team,opp_id,opp_abb,opp_team,opp_name,week_num,qualified
9348,2023,Regular,401547559,12,Week 12,ATL,4239086,D. Ridder,23.0,37.4,...,Ridder,Desmond Ridder,https://a.espncdn.com/i/headshots/nfl/players/...,Falcons,18,NO,New Orleans Saints,Saints,12,True
8719,2022,Regular,401437836,10,Week 10,CLE,2578570,J. Brissett,14.0,62.0,...,Brissett,Jacoby Brissett,https://a.espncdn.com/i/headshots/nfl/players/...,Browns,15,MIA,Miami Dolphins,Dolphins,10,True


In [20]:
# Rename columns
qbr_df = qbr_df.rename(columns={'team_abb':'posteam', 'opp_abb':'defteam'})

# Drop column
# qbr_df.drop(['game_id'], axis=1, inplace=True)

# Rename columns
qbr_df = qbr_df.rename(columns={'qb_plays': 'plays'})

# Create sequential week column
qbr_df['week'] = np.where((qbr_df['season'] < 2021) & (qbr_df['season_type'] == 'Playoffs'), (17 + qbr_df['game_week']),
                 np.where((qbr_df['season'] >= 2021) & (qbr_df['season_type'] == 'Playoffs'), (18 + qbr_df['game_week']),
                 qbr_df['game_week']))

# Modify team codes to match schedule
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'OAK', 'LV', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'OAK', 'LV', qbr_df['defteam'])
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'LAR', 'LA', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'LAR', 'LA', qbr_df['defteam'])
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'WSH', 'WAS', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'WSH', 'WAS', qbr_df['defteam'])

# Impute incorrect values
qbr_df['week'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'LV') & (qbr_df['defteam'] == 'BAL') & (qbr_df['week'] == 1), 12, qbr_df['week'])
qbr_df['week'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'BAL') & (qbr_df['defteam'] == 'LV') & (qbr_df['week'] == 1), 12, qbr_df['week'])
qbr_df['defteam'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'LV') & (qbr_df['defteam'] == 'CHI') & (qbr_df['week'] == 9), 'SF', qbr_df['defteam'])
qbr_df['defteam'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'CHI') & (qbr_df['defteam'] == 'LV') & (qbr_df['week'] == 9), 'SF', qbr_df['defteam'])

# Create both combinations of game_id
qbr_df['game_id_1'] = qbr_df['season'].astype(str) + '_' + qbr_df['week'].astype(str).str.zfill(2) + '_' + qbr_df['posteam'] + '_' + qbr_df['defteam']
qbr_df['game_id_2'] = qbr_df['season'].astype(str) + '_' + qbr_df['week'].astype(str).str.zfill(2) + '_' + qbr_df['defteam'] + '_' + qbr_df['posteam']

# Reduce dataframe dimensions
qbr_df = qbr_df[['game_id_1','game_id_2','season','week','posteam','rank','pts_added','plays','qbr_raw','qbr_total','exp_sack','sack','pass','run']]

# Merge with schedule dataframe to get game_id
qbr_df = pd.merge(qbr_df, game_df, how='left', left_on=['game_id_1'], right_on = ['game_id'])
qbr_df = qbr_df.rename(columns={'game_id':'game_id1',
                                'game_alt_id':'game_alt_id1',
                                'gameday':'gameday1',
                                'weekday':'weekday1',
                                'gametime':'gametime1',
                                'under_odds':'under_odds1',
                                'over_odds':'over_odds1',
                                'away_rest':'away_rest1',
                                'home_rest':'home_rest1',
                                })

qbr_df = pd.merge(qbr_df, game_df, how='left', left_on=['game_id_2'], right_on = ['game_id'])
qbr_df = qbr_df.rename(columns={'game_id':'game_id2',
                                'game_alt_id':'game_alt_id2',
                                'gameday':'gameday2',
                                'weekday':'weekday2',
                                'gametime':'gametime2',
                                'under_odds':'under_odds2',
                                'over_odds':'over_odds2',
                                'away_rest':'away_rest2',
                                'home_rest':'home_rest2',
                                })

# Combine columns
qbr_df['game_id'] = np.where(qbr_df['game_id1'].isnull(), qbr_df['game_id2'], qbr_df['game_id1'])
#qbr_df['game_alt_id'] = np.where(qbr_df['game_alt_id1'].isnull(), qbr_df['game_alt_id2'], qbr_df['game_alt_id1'])
qbr_df['gameday'] = np.where(qbr_df['gameday1'].isnull(), qbr_df['gameday2'], qbr_df['gameday1'])
qbr_df['weekday'] = np.where(qbr_df['weekday1'].isnull(), qbr_df['weekday2'], qbr_df['weekday1'])

qbr_df['gametime'] = np.where(qbr_df['gametime1'].isnull(), qbr_df['gametime2'], qbr_df['gametime1'])
qbr_df['under_odds'] = np.where(qbr_df['under_odds1'].isnull(), qbr_df['under_odds2'], qbr_df['under_odds1'])
qbr_df['over_odds'] = np.where(qbr_df['over_odds1'].isnull(), qbr_df['over_odds2'], qbr_df['over_odds1'])
qbr_df['away_rest'] = np.where(qbr_df['away_rest1'].isnull(), qbr_df['away_rest2'], qbr_df['away_rest1'])
qbr_df['home_rest'] = np.where(qbr_df['home_rest1'].isnull(), qbr_df['home_rest2'], qbr_df['home_rest1'])

qbr_df['game_id'].fillna('NULL', inplace = True)

qbr_df.drop(['game_id_1','game_id_2','game_id1','game_id2','gameday1','gameday2','weekday1','weekday2','gametime1','gametime2','under_odds1','under_odds2','over_odds1','over_odds2','away_rest1','away_rest2','home_rest1','home_rest2'], axis=1, inplace=True) # 'game_alt_id1','game_alt_id2',

qbr_df.shape

(1108, 22)

In [21]:
# Sort dataframe and drop quarterback with fewer passes during week
qbr_df = qbr_df.sort_values(by=['season','posteam','week','plays'], ascending=True)

qbr_df = qbr_df.drop_duplicates(['season','posteam','week'], keep='last')

qbr_df.shape

(1081, 22)

In [22]:
# Create rolling average of quarterback statistics
qbr_df['qb_rank'] = qbr_df.groupby(['season',
                                    'posteam'])['rank'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

qbr_df['qb_pts'] = qbr_df.groupby(['season',
                                    'posteam'])['pts_added'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

qbr_df['qb_plays'] = qbr_df.groupby(['season',
                                    'posteam'])['plays'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_qbr_raw'] = qbr_df.groupby(['season',
                                    'posteam'])['qbr_raw'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_qbr'] = qbr_df.groupby(['season',
                                    'posteam'])['qbr_total'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
# qbr_df['qb_sack'] = qbr_df.groupby(['season',
#                                     'posteam'])['sack'].transform(lambda x: x.rolling(window=rolling_game_window,
#                                                                                       min_periods=1,
#                                                                                       closed='left',
#                                                                                       center=False).mean())
# qbr_df['qb_exp_sack'] = qbr_df.groupby(['season',
#                                     'posteam'])['exp_sack'].transform(lambda x: x.rolling(window=rolling_game_window,
#                                                                                       min_periods=1,
#                                                                                       closed='left',
#                                                                                       center=False).mean())
qbr_df['qb_pass'] = qbr_df.groupby(['season',
                                    'posteam'])['pass'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_run'] = qbr_df.groupby(['season',
                                    'posteam'])['run'].transform(lambda x: x.rolling(window=rolling_game_window,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

# Sort dataframe by gameday and posteam
qbr_df = qbr_df.sort_values(by=['gameday','posteam'], ascending=True)

# Print column names
qbr_df.drop(['game_id','season','week','rank','pts_added','plays','qbr_raw','qbr_total','exp_sack','sack','pass','run','gameday','weekday','gametime','under_odds','over_odds','away_rest','home_rest'],  axis=1, inplace=True)

In [23]:
# Impute missing data with median values
qbr_df['qb_rank'] = qbr_df['qb_rank'].fillna(qbr_df.qb_rank.median())
qbr_df['qb_pts'] = qbr_df['qb_pts'].fillna(qbr_df.qb_pts.median())
qbr_df['qb_plays'] = qbr_df['qb_plays'].fillna(qbr_df.qb_plays.median())
qbr_df['qb_qbr_raw'] = qbr_df['qb_qbr_raw'].fillna(qbr_df.qb_qbr_raw.median())
qbr_df['qb_qbr'] = qbr_df['qb_qbr'].fillna(qbr_df.qb_qbr.median())
# qbr_df['qb_sack'] = qbr_df['qb_sack'].fillna(qbr_df.qb_sack.median())
# qbr_df['qb_exp_sack'] = qbr_df['qb_exp_sack'].fillna(qbr_df.qb_exp_sack.median())
qbr_df['qb_pass'] = qbr_df['qb_pass'].fillna(qbr_df.qb_pass.median())
qbr_df['qb_run'] = qbr_df['qb_run'].fillna(qbr_df.qb_run.median())

qbr_df.shape

(1081, 10)

In [24]:
print('team count:', qbr_df.posteam.nunique())

team count: 32


In [25]:
qbr_df.head(3)

,posteam,game_alt_id1,game_alt_id2,qb_rank,qb_pts,qb_plays,qb_qbr_raw,qb_qbr,qb_pass,qb_run
1,BUF,LA_BUF_2022_09_08,NaN,15.75,0.2,41.5,52.1,52.25,3.025,0.375
26,LA,NaN,LA_BUF_2022_09_08,15.75,0.2,41.5,52.1,52.25,3.025,0.375
14,ARI,NaN,ARI_KC_2022_09_11,15.75,0.2,41.5,52.1,52.25,3.025,0.375


## NFL GAME ROSTERS

## NFL DEPTH CHART DATA

## NFL SNAP COUNT DATA

In [26]:
sc_data = nfl.import_snap_counts(years=seasons)

# Clean dataframe using nfl-data-api cleaning function
sc_df = nfl.clean_nfl_data(sc_data)

sc_df.shape

(51643, 16)

In [27]:
print('team count:', sc_df.team.nunique())

team count: 32


In [28]:
sc_df.sample(2)

,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct
9242,2023_07_JAX_NO,202310190nor,2023,REG,7,Tommy Kraemer,KraeTo02,G,NO,JAX,0.0,0.0,0.0,0.0,5.0,0.16
9235,2023_07_JAX_NO,202310190nor,2023,REG,7,D'Marco Jackson,JackDM00,LB,NO,JAX,0.0,0.0,0.0,0.0,26.0,0.81


## NFL COMBINE DATA

In [29]:
co_data = nfl.import_combine_data(years=draft_years)

# Clean dataframe using nfl-data-api cleaning function
co_df = nfl.clean_nfl_data(co_data)

co_df.shape

(7999, 18)

In [30]:
co_df['pfr_id'].fillna('NULL', inplace = True)
co_df = co_df.loc[co_df['pfr_id'] != 'NULL']

co_df = co_df.rename(columns={'pfr_id':'pfr_player_id',
                              'season':'combine',
                              'player_name':'comb_name',
                              'school':'comb_school',
                              'ht':'comb_ht',
                              'wt':'comb_wt',
                              'pos':'comb_pos',
                              'forty':'comb_forty',
                              'bench':'comb_bench',
                              'vertical':'comb_vert',
                              'broad_jump':'comb_broad',
                              'cone':'comb_cone',
                              'shuttle':'comb_shut',
                              })

co_df.drop(['draft_year','draft_team','draft_round','draft_ovr','cfb_id'], axis=1, inplace=True)

co_df.shape

(6606, 13)

In [31]:
# Convert height to inches
co_df['comb_ht'].fillna('NULL', inplace = True)
co_df['comb_ht'] = np.where(co_df['comb_ht'] == 'NULL', '0-0', co_df['comb_ht'])
co_df['comb_ht'] = (co_df['comb_ht'].astype(str).str[0]).astype(float) * 12 + (co_df['comb_ht'].astype(str).str[2]).astype(float)

In [32]:
co_df.sample(1)

,combine,pfr_player_id,comb_name,comb_pos,comb_school,comb_ht,comb_wt,comb_forty,comb_bench,comb_vert,comb_broad,comb_cone,comb_shut
887,2002,RichKr20,Kris Richard,CB,USC,61.0,186.0,4.47,10.0,39.5,120.0,NaN,4.15


## NFL DRAFT DATA

In [33]:
dr_data = nfl.import_draft_picks(years=draft_years)

# Clean dataframe using nfl-data-api cleaning function
dr_df = nfl.clean_nfl_data(dr_data)

dr_df.shape

(6130, 36)

In [34]:
dr_df['team'] = np.where(dr_df['team'] == 'GNB', 'GB', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'KAN', 'KC', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'RAM', 'LA', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'STL', 'LA', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'SDG', 'LAC', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'RAI', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'OAK', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'LVR', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'LAR', 'LV', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'NWE', 'NE', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'NOR', 'NO', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'SFO', 'SF', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'TAM', 'TB', dr_df['team'])

print('team count:', dr_df.team.nunique())

team count: 32


In [35]:
# Calculate years in the NFL
dr_df['to'].fillna((dr_df['season']-1), inplace=True)

dr_df['nfl_years'] = dr_df['to'] - dr_df['season'] + 1

In [36]:
# Convert HOF to binary
dr_df['hof'] = np.where(dr_df['hof'] == True, 1, 0)

In [37]:
dr_df['pfr_player_id'].fillna('NULL', inplace = True)
dr_df = dr_df.loc[dr_df['pfr_player_id'] != 'NULL']

dr_df = dr_df.rename(columns={'season':'draft',
                              'team':'draft_team',
                              'pfr_player_name':'player_name',
                              'probowls':'pro_bowls'})

dr_df.drop(['cfb_player_id','gsis_id','w_av','car_av','dr_av','pass_attempts','pass_completions','pass_yards','pass_tds','pass_ints','rush_atts','rush_yards','rush_tds','receptions','rec_yards','rec_tds','def_solo_tackles','def_ints','def_sacks','to'], axis=1, inplace=True)

dr_df.shape

(5874, 17)

## MERGE DRAFT AND COMBINE DATA

In [38]:
print('combine records:', co_df.shape[0])
print('draft records:', dr_df.shape[0])

combine records: 6606
draft records: 5874


In [39]:
draft_df = pd.merge(dr_df, co_df, how='inner', left_on=['pfr_player_id'], right_on = ['pfr_player_id'])

draft_df.shape

(4967, 29)

In [40]:
draft_df = draft_df.loc[(draft_df['category'] != 'P') & 
                        (draft_df['category'] != 'K') & 
                        (draft_df['category'] != 'LS')]

draft_df.shape

(4892, 29)

In [41]:
# Impute missing values
draft_df['age'] = draft_df.age.fillna(draft_df.groupby('category').age.transform('median'))
draft_df['comb_ht'] = draft_df.comb_ht.fillna(draft_df.groupby('category').comb_ht.transform('median'))
draft_df['comb_wt'] = draft_df.comb_wt.fillna(draft_df.groupby('category').comb_wt.transform('median'))
draft_df['comb_forty'] = draft_df.comb_forty.fillna(draft_df.groupby('category').comb_forty.transform('median'))
draft_df['comb_bench'] = draft_df.comb_bench.fillna(draft_df.groupby('category').comb_bench.transform('median'))
draft_df['comb_vert'] = draft_df.comb_vert.fillna(draft_df.groupby('category').comb_vert.transform('median'))
draft_df['comb_broad'] = draft_df.comb_broad.fillna(draft_df.groupby('category').comb_broad.transform('median'))
draft_df['comb_cone'] = draft_df.comb_cone.fillna(draft_df.groupby('category').comb_cone.transform('median'))
draft_df['comb_shut'] = draft_df.comb_shut.fillna(draft_df.groupby('category').comb_shut.transform('median'))
draft_df['games'].fillna(0, inplace = True)

In [42]:
# Count missing data
missing = draft_df.isnull().sum()
missing

draft              0
round              0
pick               0
draft_team         0
pfr_player_id      0
player_name        0
hof                0
position           0
category           0
side               0
college            0
age                0
allpro             0
pro_bowls          0
seasons_started    0
games              0
nfl_years          0
combine            0
comb_name          0
comb_pos           0
comb_school        0
comb_ht            0
comb_wt            0
comb_forty         0
comb_bench         0
comb_vert          0
comb_broad         0
comb_cone          0
comb_shut          0
dtype: int64

In [43]:
draft_df.sample(5)

,draft,round,pick,draft_team,pfr_player_id,player_name,hof,position,category,side,...,comb_pos,comb_school,comb_ht,comb_wt,comb_forty,comb_bench,comb_vert,comb_broad,comb_cone,comb_shut
1970,2010,1,8,LV,McClRo00,Rolando McClain,0,LB,LB,D,...,ILB,Alabama,75.0,254.0,4.68,24.0,34.5,119.0,7.10,4.28
3040,2014,7,240,LAC,ReesTe00,Tevin Reese,0,WR,WR,O,...,WR,Baylor,61.0,163.0,4.46,15.0,41.0,132.0,6.63,4.18
1501,2007,5,172,LAC,NaanLe00,Legedu Naanee,0,WR,WR,O,...,WR,Boise State,74.0,225.0,4.41,19.0,40.0,119.0,6.95,4.20
4893,2023,4,134,MIN,WardJa00,Jay Ward,0,S,DB,D,...,CB,LSU,73.0,188.0,4.55,16.0,34.5,132.0,6.94,4.17
2260,2011,3,88,NO,PatrJo00,Johnny Patrick,0,DB,DB,D,...,CB,Louisville,61.0,191.0,4.59,13.0,32.0,110.0,6.94,4.16


In [44]:
# Export binary classifier dataframe
draft_df.to_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_draft_data.csv', index=None, header=True)

## MERGE PLAY-BY-PLAY WITH SUPPLEMENTARY DATA

In [45]:
df = play_df.copy()

# _______________
## PRELIMINARY PLAY-BY-PLAY FILTERS

In [46]:
# Exclude columns with the following records
df = df.loc[~(df['desc'].str.startswith("END |END_|Two-Minute|Two Minute|Two minute|Two-minute|Two minute|Two-min"))]

# Exclude columns with specific keywords
df = df.loc[~(df['desc'].str.contains("Captains:|CAPTAINS:|Captians:|Captains #|Captians #"))]
df = df.loc[~(df['desc'].str.contains("Two-Minute Warning"))]
df = df.loc[~(df['desc'].str.contains("game has been suspended|game has resumed|Game was resumed"))]
df = df.loc[~(df['desc'].str.contains("Game delayed|game delayed|Game suspended|Game was resumed"))]
df = df.loc[~(df['desc'].str.contains("no play run|Humidity|weather delay|severe weather"))]
df = df.loc[df['down'] >= 1.0]
df = df.loc[df['special_teams_play'] == 0]
df = df.loc[df['season_type'] != 'PRE']
df = df.loc[~(df['play_type'].isin(['punt','kickoff','extra_point','field_goal','qb_kneel','qb_spike']))]
df = df.loc[df['aborted_play'] == 0]
df = df.loc[df['play_deleted'] == 0]
df = df.loc[~df['drive'].isnull()]
df = df.loc[~(df['offense_personnel'].astype(str).str.contains("LS"))]

df.shape

(68889, 391)

In [47]:
# Print a list of columns with only one value and then drop them
single_value_columns = df.loc[:,df.nunique() == 1].columns
df = df.loc[:,df.nunique() > 1]

print('Single value columns:', single_value_columns)
print('Remaining columns:', df.shape[1])

Single value columns: Index(['quarter_end', 'qb_kneel', 'qb_spike', 'extra_point_prob',
       'two_point_conversion_prob', 'punt_blocked', 'punt_inside_twenty',
       'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed',
       'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone',
       'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch',
       'own_kickoff_recovery', 'own_kickoff_recovery_td',
       'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt',
       'kickoff_attempt', 'punt_attempt', 'tackle_with_assist_2_player_id',
       'tackle_with_assist_2_player_name', 'tackle_with_assist_2_team',
       'defensive_two_point_attempt', 'defensive_two_point_conv',
       'defensive_extra_point_attempt', 'defensive_extra_point_conv',
       'play_deleted', 'special_teams_play', 'aborted_play', 'special'],
      dtype='object')
Remaining columns: 326


In [48]:
# Convert missing values to Null
df = df.fillna(value=np.nan)

In [49]:
# Convert field to datetime format
df['game_date']= pd.to_datetime(df['game_date'])

# Sort dataframe
df = df.sort_values(by=['posteam','game_date','play'], ascending=True)

df.shape

(68889, 326)

## PERSONNEL FEATURES

In [50]:
df['report_eligible'] = np.where(df['desc'].str.find('as eligible')>= 0, 1, 0)

df.report_eligible.mean()

0.018827388988082276

In [51]:
# Remove commas from strings
df['offense_personnel'] = df['offense_personnel'].str.replace(',', ', ').str.replace('  ',' ').str.replace(',', '').str.strip()
df['defense_personnel'] = df['defense_personnel'].str.replace(',', ', ').str.replace('  ',' ').str.replace(',', '').str.strip()

# Impute personnel with modes
df['offense_personnel'].fillna(df['offense_personnel'].mode()[0], inplace=True)
df['defense_personnel'].fillna(df['defense_personnel'].mode()[0], inplace=True)

In [52]:
# Offensive personnel counts
df['off_rb_count'] = df['offense_personnel'].str.extract('(\d+) RB').fillna(0).astype(int)
#df['off_te_count'] = df['offense_personnel'].str.extract('(\d+) TE').fillna(0).astype(int)
df['off_wr_count'] = df['offense_personnel'].str.extract('(\d+) WR').fillna(0).astype(int)
#df['off_hb_count'] = df['off_rb_count'] + df['off_te_count']

df['off_ol_count'] = df['offense_personnel'].str.extract('(\d+) OL').fillna(5).astype(int)
df['off_ol_count'] = np.where((df['off_ol_count'] == 5) & (df['report_eligible'] == 1), 6, df['off_ol_count']) 

# Defensive personnel counts
df['def_dl_count'] = df['defense_personnel'].str.extract('(\d+) DL').fillna(0).astype(int)
#df['def_lb_count'] = df['defense_personnel'].str.extract('(\d+) LB').fillna(0).astype(int)
df['def_db_count'] = df['defense_personnel'].str.extract('(\d+) DB').fillna(0).astype(int)

# Calculate personnel ratios
df['wr_to_db_ratio'] = df['off_wr_count'] / df['def_db_count']
#df['hb_to_lb_ratio'] = df['off_hb_count'] / df['def_lb_count']
df['ol_to_dl_ratio'] = df['off_ol_count'] / df['def_dl_count']

# Convert inf to 0
df['wr_to_db_ratio'] = np.where((df['off_wr_count'] > 0) & (df['def_db_count'] == 0), df['off_wr_count'], 
                                np.where((df['def_db_count'] > 0) & (df['off_wr_count'] == 0), df['def_db_count'],df['wr_to_db_ratio']))

#df['hb_to_lb_ratio'] = np.where((df['off_hb_count'] > 0) & (df['def_lb_count'] == 0), df['off_hb_count'], np.where((df['def_lb_count'] > 0) & (df['off_hb_count'] == 0), df['def_lb_count'], df['hb_to_lb_ratio']))

df['ol_to_dl_ratio'] = np.where((df['off_ol_count'] > 0) & (df['def_dl_count'] == 0), df['off_ol_count'], np.where((df['def_dl_count'] > 0) & (df['off_ol_count'] == 0), df['def_dl_count'], df['ol_to_dl_ratio']))

# Convert inf values to 0
df['wr_to_db_ratio'] = np.where(df['wr_to_db_ratio'] == np.inf, 0, df['wr_to_db_ratio'])
#df['hb_to_lb_ratio'] = np.where(df['hb_to_lb_ratio'] == np.inf, 0, df['hb_to_lb_ratio'])
df['ol_to_dl_ratio'] = np.where(df['ol_to_dl_ratio'] == np.inf, 0, df['ol_to_dl_ratio'])

# Impute missing values
df['wr_to_db_ratio'].fillna(df['wr_to_db_ratio'].median(), inplace=True)
# df['hb_to_lb_ratio'].fillna(df['hb_to_lb_ratio'].median(), inplace=True)
df['ol_to_dl_ratio'].fillna(df['ol_to_dl_ratio'].median(), inplace=True)

# Drop personnel counts
df.drop(['off_rb_count','off_wr_count','off_ol_count','def_dl_count','def_db_count'], axis=1, inplace=True)

## MODIFY DATAFRAME

In [53]:
# Impute missing values
df['time_of_day'].fillna(method='ffill', inplace=True)
df['quarter_seconds_remaining'].fillna(method='bfill', inplace=True)
df['half_seconds_remaining'].fillna(method='bfill', inplace=True)
df['game_seconds_remaining'].fillna(method='bfill', inplace=True)
df['wp'].fillna(method='bfill', inplace=True)
df['def_wp'].fillna(method='bfill', inplace=True)


In [54]:
# Strip all whitespace from strings
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Strips all objects in dataframe
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [55]:
# Impute missing values
df['play_type'].fillna('no_play', inplace = True)
df['play_type_nfl'].fillna('NO_PLAY', inplace = True)
df['sp'].fillna(0, inplace = True)
df['qtr'].fillna(0, inplace = True)
df['goal_to_go'].fillna(0, inplace = True)
df['ydstogo'].fillna(0, inplace = True) 
df['ydsnet'].fillna(0, inplace = True)
df['shotgun'].fillna(0, inplace = True)
df['no_huddle'].fillna(0, inplace = True)
df['qb_dropback'].fillna(0, inplace = True)
df['qb_scramble'].fillna(0, inplace = True)
df['yards_after_catch'].fillna(0, inplace = True)
df['home_timeouts_remaining'].fillna(0, inplace = True)
df['away_timeouts_remaining'].fillna(0, inplace = True)
df['timeout'].fillna(0, inplace = True)
df['posteam_timeouts_remaining'].fillna(0, inplace = True) 
df['defteam_timeouts_remaining'].fillna(0, inplace = True)
df['total_home_score'].fillna(0, inplace = True)
df['total_away_score'].fillna(0, inplace = True)
df['posteam_score'].fillna(0, inplace = True) 
df['defteam_score'].fillna(0, inplace = True)
df['score_differential'].fillna(0, inplace = True)
df['posteam_score_post'].fillna(0, inplace = True)
df['defteam_score_post'].fillna(0, inplace = True)
df['score_differential_post'].fillna(0, inplace = True)
df['first_down_rush'].fillna(0, inplace = True)
df['first_down_pass'].fillna(0, inplace = True)
df['first_down_penalty'].fillna(0, inplace = True)
df['third_down_converted'].fillna(0, inplace = True)
df['third_down_failed'].fillna(0, inplace = True)
df['fourth_down_converted'].fillna(0, inplace = True)
df['fourth_down_failed'].fillna(0, inplace = True)
df['incomplete_pass'].fillna(0, inplace = True)
df['touchback'].fillna(0, inplace = True)
df['interception'].fillna(0, inplace = True)
df['fumble_forced'].fillna(0, inplace = True)
df['fumble_not_forced'].fillna(0, inplace = True)
df['fumble_out_of_bounds'].fillna(0, inplace = True)
df['solo_tackle'].fillna(0, inplace = True)
df['penalty'].fillna(0, inplace = True)
df['tackled_for_loss'].fillna(0, inplace = True)
df['fumble'].fillna(0, inplace = True)
df['fumble_lost'].fillna(0, inplace = True)
df['qb_hit'].fillna(0, inplace = True)
df['rush_attempt'].fillna(0, inplace = True)
df['pass_attempt'].fillna(0, inplace = True)
df['sack'].fillna(0, inplace = True)
df['safety'].fillna(0, inplace = True)
df['touchdown'].fillna(0, inplace = True)
df['pass_touchdown'].fillna(0, inplace = True)
df['rush_touchdown'].fillna(0, inplace = True)
df['return_touchdown'].fillna(0, inplace = True)
df['complete_pass'].fillna(0, inplace = True)
df['assist_tackle'].fillna(0, inplace = True)
df['lateral_reception'].fillna(0, inplace = True)
df['lateral_rush'].fillna(0, inplace = True)
df['lateral_return'].fillna(0, inplace = True)
df['lateral_recovery'].fillna(0, inplace = True)
df['passing_yards'].fillna(0, inplace = True)
df['receiving_yards'].fillna(0, inplace = True)
df['rushing_yards'].fillna(0, inplace = True)
df['lateral_receiving_yards'].fillna(0, inplace = True)
df['lateral_rushing_yards'].fillna(0, inplace = True)
df['tackle_with_assist'].fillna(0, inplace = True)
df['return_yards'].fillna(0, inplace = True)
df['replay_or_challenge'].fillna(0, inplace = True)
df['series_success'].fillna(0, inplace = True)
df['order_sequence'].fillna(0, inplace = True)
df['fixed_drive'].fillna(0, inplace = True)
df['drive_play_count'].fillna(0, inplace = True)
df['drive_first_downs'].fillna(0, inplace = True)
df['drive_inside20'].fillna(0, inplace = True)
df['drive_ended_with_score'].fillna(0, inplace = True)
df['drive_quarter_start'].fillna(0, inplace = True)
df['drive_quarter_end'].fillna(0, inplace = True)
df['drive_yards_penalized'].fillna(0, inplace = True)
df['drive_play_id_started'].fillna(0, inplace = True)
df['drive_play_id_ended'].fillna(0, inplace = True)
df['success'].fillna(0, inplace = True)
df['pass'].fillna(0, inplace = True)
df['rush'].fillna(0, inplace = True)
df['first_down'].fillna(0, inplace = True)
df['play'].fillna(0, inplace = True)
df['out_of_bounds'].fillna(0, inplace = True)
df['home_opening_kickoff'].fillna(0, inplace = True)
df['fumble_recovery_1_yards'].fillna(0, inplace = True)
df['fumble_recovery_2_yards'].fillna(0, inplace = True)
df['penalty_yards'].fillna(0, inplace = True)

In [56]:
# Convert to integer
df['play_id'] = df['play_id'].astype(int)
df['quarter_seconds_remaining'] = df['quarter_seconds_remaining'].astype(int)
df['half_seconds_remaining'] = df['half_seconds_remaining'].astype(int)
df['game_seconds_remaining'] = df['game_seconds_remaining'].astype(int)
df['sp'] = df['sp'].astype(int)
df['qtr'] = df['qtr'].astype(int)
df['goal_to_go'] = df['goal_to_go'].astype(int)
df['ydstogo'] = df['ydstogo'].astype(int)
df['shotgun'] = df['shotgun'].astype(int)
df['no_huddle'] = df['no_huddle'].astype(int)
df['qb_dropback'] = df['qb_dropback'].astype(int)
df['qb_scramble'] = df['qb_scramble'].astype(int)
df['yards_after_catch'] = df['yards_after_catch'].astype(int)
df['home_timeouts_remaining'] = df['home_timeouts_remaining'].astype(int)
df['away_timeouts_remaining'] = df['away_timeouts_remaining'].astype(int)
df['timeout'] = df['timeout'].astype(int)
df['posteam_timeouts_remaining'] = df['posteam_timeouts_remaining'].astype(int)
df['defteam_timeouts_remaining'] = df['defteam_timeouts_remaining'].astype(int)
df['total_home_score'] = df['total_home_score'].astype(int)
df['total_away_score'] = df['total_away_score'].astype(int)
df['posteam_score'] = df['posteam_score'].astype(int)
df['defteam_score'] = df['defteam_score'].astype(int)
df['score_differential'] = df['score_differential'].astype(int)
df['posteam_score_post'] = df['posteam_score_post'].astype(int)
df['defteam_score_post'] = df['defteam_score_post'].astype(int)
df['score_differential_post'] = df['score_differential_post'].astype(int)
df['first_down_rush'] = df['first_down_rush'].astype(int)
df['first_down_pass'] = df['first_down_pass'].astype(int)
df['first_down_penalty'] = df['first_down_penalty'].astype(int)
df['third_down_converted'] = df['third_down_converted'].astype(int)
df['third_down_failed'] = df['third_down_failed'].astype(int)
df['fourth_down_converted'] = df['fourth_down_converted'].astype(int)
df['fourth_down_failed'] = df['fourth_down_failed'].astype(int)
df['incomplete_pass'] = df['incomplete_pass'].astype(int)
df['touchback'] = df['touchback'].astype(int)
df['interception'] = df['interception'].astype(int)
df['fumble_forced'] = df['fumble_forced'].astype(int)
df['fumble_not_forced'] = df['fumble_not_forced'].astype(int)
df['fumble_out_of_bounds'] = df['fumble_out_of_bounds'].astype(int)
df['solo_tackle'] = df['solo_tackle'].astype(int)
df['safety'] = df['safety'].astype(int)
df['penalty'] = df['penalty'].astype(int)
df['tackled_for_loss'] = df['tackled_for_loss'].astype(int)
df['fumble_lost'] = df['fumble_lost'].astype(int)
df['qb_hit'] = df['qb_hit'].astype(int)
df['rush_attempt'] = df['rush_attempt'].astype(int)
df['pass_attempt'] = df['pass_attempt'].astype(int)
df['sack'] = df['sack'].astype(int)
df['touchdown'] = df['touchdown'].astype(int)
df['rush_touchdown'] = df['rush_touchdown'].astype(int)
df['pass_touchdown'] = df['pass_touchdown'].astype(int)
df['return_touchdown'] = df['return_touchdown'].astype(int)
df['fumble'] = df['fumble'].astype(int)
df['complete_pass'] = df['complete_pass'].astype(int)
df['assist_tackle'] = df['assist_tackle'].astype(int)
df['lateral_reception'] = df['lateral_reception'].astype(int)
df['lateral_rush'] = df['lateral_rush'].astype(int)
df['lateral_return'] = df['lateral_return'].astype(int)
df['lateral_recovery'] = df['lateral_recovery'].astype(int)
df['passing_yards'] = df['passing_yards'].astype(int)
df['receiving_yards'] = df['receiving_yards'].astype(int)
df['rushing_yards'] = df['rushing_yards'].astype(int)
df['lateral_receiving_yards'] = df['lateral_receiving_yards'].astype(int)
df['lateral_rushing_yards'] = df['lateral_rushing_yards'].astype(int)
df['tackle_with_assist'] = df['tackle_with_assist'].astype(int)
df['return_yards'] = df['return_yards'].astype(int)
df['replay_or_challenge'] = df['replay_or_challenge'].astype(int)
df['series'] = df['series'].astype(int)
df['series_success'] = df['series_success'].astype(int)
df['order_sequence'] = df['order_sequence'].astype(int)
df['fixed_drive'] = df['fixed_drive'].astype(int)
df['drive_play_count'] = df['drive_play_count'].astype(int)
df['drive_first_downs'] = df['drive_first_downs'].astype(int)
df['drive_inside20'] = df['drive_inside20'].astype(int)
df['drive_ended_with_score'] = df['drive_ended_with_score'].astype(int)
df['drive_quarter_start'] = df['drive_quarter_start'].astype(int)
df['drive_quarter_end'] = df['drive_quarter_end'].astype(int)
df['drive_yards_penalized'] = df['drive_yards_penalized'].astype(int)
df['drive_play_id_started'] = df['drive_play_id_started'].astype(int)
df['drive_play_id_ended'] = df['drive_play_id_ended'].astype(int)
df['success'] = df['success'].astype(int)
df['pass'] = df['pass'].astype(int)
df['rush'] = df['rush'].astype(int)
df['first_down'] = df['first_down'].astype(int)
df['play'] = df['play'].astype(int)
df['out_of_bounds'] = df['out_of_bounds'].astype(int)
df['home_opening_kickoff'] = df['home_opening_kickoff'].astype(int)
df['fumble_recovery_1_yards'] = df['fumble_recovery_1_yards'].astype(int)
df['fumble_recovery_2_yards'] = df['fumble_recovery_2_yards'].astype(int)
df['penalty_yards'] = df['penalty_yards'].astype(int)
# df['home_rest'] = df['home_rest'].astype(int)
# df['away_rest'] = df['away_rest'].astype(int)
df.shape

(68889, 329)

In [57]:
counts = df.play_type_nfl.value_counts(normalize=False)
counts

play_type_nfl
PASS                            36717
RUSH                            28950
SACK                             2764
INTERCEPTION                      441
FUMBLE_RECOVERED_BY_OPPONENT       15
UNSPECIFIED                         2
Name: count, dtype: int64

In [58]:
counts = df.play_type.value_counts(normalize=False)
counts

play_type
pass    39937
run     28952
Name: count, dtype: int64

## IMPUTE PLAY TYPE

In [59]:
df1 = df.copy()

df1.shape

(68889, 329)

In [60]:
# Modify play type
df1['desc'] = df1['desc'].str.strip().str.lower()

# QB Scramble
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('scrambles ')>= 0), 'qb_scramble', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'run') & (df1['desc'].str.find('scrambles ')>= 0), 'qb_scramble', df1['play_type'])

# Kickoffs
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('kicks')>= 0), 'kickoff', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('kick formation')>= 0), 'kickoff', df1['play_type'])

# Field goals
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('field goal')>= 0), 'field_goal', df1['play_type'])

# Punts
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' punts')>= 0), 'punt', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' punt is')>= 0), 'punt', df1['play_type'])

# Extra points (PATs)
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' extra point')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'extra_point', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('two-point conversion')>= 0), 'two_point', df1['play_type'])

# Kneel
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' kneels')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'qb_kneel', df1['play_type'])

# QB spikes
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' spiked')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'qb_spike', df1['play_type'])

# Passes
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass incomplete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass complete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass short')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass deep')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass to')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('sacked')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass intended')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass intercepted')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass incomplete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass complete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass short')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass deep')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass to')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('sacked')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass intended')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass intercepted')>= 0), 'pass', df1['play_type'])

# Runs
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('up the middle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('end around')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('up the middle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('end around')>= 0), 'run', df1['play_type'])

In [61]:
# Penalties
df1['play_type'] = np.where(df1['desc'].str.find('false start')>= 0, 'penalty', df1['play_type'])
df1['play_type'] = np.where(df1['desc'].str.find('offensive delay of game')>= 0, 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty') >= 0) & (df1['desc'].str.find('neutral zone')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('delay of kickoff')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('too many men')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('encroachment')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('12 on-field')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('unsportsmanlike')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('interference')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('holding')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('illegal')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('offside')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('roughness')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('chop block')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('tripping')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('roughing')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('face mask')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('personal foul')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('disqualification')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('taunting')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('intentional grounding')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('play over the down')>= 0), 'penalty', df1['play_type'])

# Replay reviews
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('replay was upheld')>= 0), 'replay_review', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('replay assistant')>= 0), 'replay_review', df1['play_type'])

# Timeouts
df1['play_type'] = np.where((df1['desc'].str.find('timeout #')>= 0) & (df1['desc'].str.find('passer_jersey_number')>= 0) &  (df1['desc'].str.find('rusher_jersey_number')>= 0), 'timeout', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('timeout at')>= 0) & (df1['desc'].str.find('passer_jersey_number')>= 0) &  (df1['desc'].str.find('rusher_jersey_number')>= 0), 'timeout', df1['play_type'])

# Clock runoff (no play)
df1['play_type'] = np.where(df1['desc'].str.find('end of quarter due to 10 second clock run-off')>= 0, 'clock_runoff', df1['play_type'])

# Replay down (no play)
df1['play_type'] = np.where(df1['desc'].str.find('play over the down')>= 0, 'replay_down', df1['play_type'])
df1['play_type'] = np.where(df1['desc'].str.find('play the down')>= 0, 'replay_down', df1['play_type'])

In [62]:
df1['play_type_nfl'] = np.where(df1['play_type'] == 'run', 'RUSH',
                                np.where(df1['play_type'] == 'penalty', 'PENALTY',
                                         np.where(df1['play_type'] == 'extra_point', 'XP_KICK',
                                                  np.where(df1['play_type'] == 'field_goal', 'FIELD_GOAL',
                                                           np.where(df1['play_type'] == 'timeout', 'TIMEOUT',
                                                                    np.where(df1['play_type'] == 'two_point', 'PAT2',
                                                                             df1['play_type_nfl']))))))

# Impute down property
df1['down'] = np.where(df1['play_type'] == 'kickoff', 0,
                       np.where(df1['play_type'] == 'extra_point', 0,
                                np.where(df1['play_type'] == 'two_point', 0,
                                         np.where(df1['play_type'] == 'injury',  df1['down'].shift(-1),
                                                  np.where(df1['play_type'] == 'replay',  df1['down'].shift(-1),
                                                           np.where(df1['play_type'] == 'timeout', df1['down'].shift(1),
                                                                    np.where(df1['play_type'] == 'penalty', df1['down'].shift(1),
                                                                             df1['down'])))))))

# Remove kickoff from drive groupings
df1['drive'] = np.where(df1['play_type'] == 'kickoff', np.nan, df1['drive'])

# Offsetting penalties
df1['offsetting_penalties'] = np.where((df1['desc'].str.find('offsetting') >= 0) & (df1['desc'].str.find('no play') >= 0), 1, 0)

# Impute missing yards_gained
df1['yards_gained'] = np.where(df1['offsetting_penalties'] == 1, 0, df1['yards_gained'])
df1['yards_gained'] = np.where(df1['play_type'] == 'timeout', 0, df1['yards_gained'])
df1['yards_gained'] = np.where((df1['play_type']=='penalty') & (df1['yards_gained']==np.nan),0,df1['yards_gained'])

In [63]:
# Impute pass location for pass play type when missing
df1['pass_location'] = np.where((df1['play_type']=='pass') & (df1['desc'].str.find('pass middle')>= 0), 'middle',
                                np.where((df1['play_type']=='pass') &  (df1['desc'].str.find('incomplete middle')>= 0), 'middle', 
                                         np.where((df1['play_type']=='pass') & (df1['desc'].str.find('pass left')>= 0), 'left',
                                                  np.where((df1['play_type']=='pass') & (df1['desc'].str.find('incomplete left')>= 0), 'left', 
                                                           np.where((df1['play_type']=='pass') & (df1['desc'].str.find('pass right')>= 0), 'right',
                                                                    np.where((df1['play_type']=='pass') & (df1['desc'].str.find('incomplete right')>= 0), 'right',
                                                                             df1['pass_location']))))))

# Impute pass length for pass play type when missing
df1['pass_length'] = np.where((df1['play_type']=='pass') & (df1['pass_length']==np.nan) & (df1['desc'].str.find('deep')>= 0), 'deep',
                              np.where((df1['play_type']=='pass') & (df1['pass_length']==np.nan) & (df1['desc'].str.find('short')>= 0), 'short',
                                       np.where((df1['play_type']=='pass') & (df1['pass_length']==np.nan) & (df1['desc'].str.find('deep, incomplete')>= 0), 'deep',
                                                np.where((df1['play_type']=='pass') & (df1['pass_length']==np.nan) & (df1['desc'].str.find('short, incomplete')>= 0), 'short',
                                                         df1['pass_length']))))

df1['intentional_grounding'] = np.where((df1['play_type']=='pass') & (df1['desc'].str.find('intentional grounding')>= 0), 1, 0)


## INITIAL PLAY FILTERS

In [64]:
df2 = df1.copy()

df2.shape

(68889, 331)

In [65]:
counts = df2.play_type_nfl.value_counts(normalize=False)
counts

play_type_nfl
PASS                            36717
RUSH                            28952
SACK                             2764
INTERCEPTION                      441
FUMBLE_RECOVERED_BY_OPPONENT       15
Name: count, dtype: int64

In [66]:
counts = df2.play_type.value_counts(normalize=False, dropna=False)
counts

play_type
pass           39937
run            26960
qb_scramble     1992
Name: count, dtype: int64

In [67]:
df2 = df2[~df2['play_type'].isin(['punt','qb_kneel','field_goal','qb_scramble','qb_spike'])] # retain penalty to capture previous penalty events

df2.shape

(66897, 331)

In [68]:
counts = df2.play_type.value_counts(normalize=False, dropna=False)
counts

play_type
pass    39937
run     26960
Name: count, dtype: int64

## FEATURE ENGINEERING

In [69]:
# Game-play sequence
df2['play_sequence_game'] = df2.groupby(['game_id'])['play_id'].cumcount() + 1

# Game-play-drive sequence
df2['play_sequence_series'] = df2.groupby(['game_id','drive']).cumcount() + 1

# Code turnover on play
df2['turnover'] = np.where((df2['fumble_lost'] == 1) | (df2['interception'] == 1), 1, 0)

In [70]:
# Normalize score_differential between 0 and 1, based on min and max values
df2['score_differential_norm'] = (df2['score_differential'] - df2['score_differential'].min()) / (df2['score_differential'].max() - df2['score_differential'].min())


In [71]:
df2['play_type_detail'] = np.where((df2['play_type']=='run') & (df2['desc'].str.find('left end')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right end')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('end around')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('left tackle')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('left guard')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('middle')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right guard')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right tackle')>= 0), 'run_outside',

                          np.where(df2['pass_length']=='deep', 'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']>10), 'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('pass deep')>= 0),'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('complete deep')>= 0), 'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['air_yards'] == np.nan) & (df2['yards_after_catch'] == 0) & (df2['passing_yards'] >= 10), 'pass_deep',

                          np.where(df2['pass_length']=='short', 'pass_short',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']<= 10), 'pass_short',
                          np.where((df2['play_type']=='pass') & (df2['air_yards'] == np.nan) & (df2['yards_after_catch'] == 0) & (df2['passing_yards'] < 10), 'pass_short',
                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('intentional grounding')>= 0), 'pass_short',

                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('sack')>= 0),'pass_sack',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']==np.nan), 'pass',

                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('aborted')>= 0),'aborted',
                          df2['play_type']))))))))))))))))))))

counts = df2.play_type_detail.value_counts(normalize=False)
counts

play_type_detail
pass_short     26244
run_inside     13950
run_outside    12986
pass_deep      10926
pass_sack       2765
run               24
pass               2
Name: count, dtype: int64

In [72]:
# Remove ® from stadium names
df2['stadium'] = df2['stadium'].str.replace('®', '').str.replace('-','').str.replace('&','').str.replace('.','').str.replace("'","").str.strip().str.lower()

# Impute nan values to NULL
df2['surface'] = np.where(df2['surface'] == '', np.nan, df2['surface'])

# Map stadiums to new values
stadium_map = {'allegiant stadium': 'grass',
               'azteca stadium': 'grass',
               'fedexfield': 'grass',
               'state farm stadium': 'grass',
               'mercedesbenz stadium': 'turf',
               'mt bank stadium': 'grass',
               'paycor stadium': 'turf',
               'metlife stadium':'turf',
               'highmark stadium':'turf',
               'bank of america stadium': 'grass',
               'soldier field': 'grass',
               'raymond james stadium': 'grass',
               'cleveland browns stadium': 'grass', 
               'firstenergy stadium': 'grass',
               'acrisure stadium': 'grass',
               'att stadium': 'turf',
               'wembley stadium': 'grass',
               'empower field at mile high': 'grass', 
               'geha field at arrowhead stadium': 'grass', 
               'ford field': 'turf',
               'tottenham hotspur stadium': 'turf',
               'nrg stadium': 'turf',
               'lucas oil stadium': 'turf',
               'everbank stadium': 'grass',
               'tiaa bank field': 'grass',
               "levis stadium": 'grass',
               'lumen field': 'turf',
               'sofi stadium': 'turf', 
               'nissan stadium': 'turf',
               'gillette stadium': 'turf',
               'us bank stadium': 'turf',
               'lincoln financial field': 'grass',
               'caesars superdome': 'turf',
               'allianz arena': 'turf',
               'hard rock stadium': 'grass',
               'lambeau field': 'grass',
               'frankfurt stadium': 'turf',
               }

df2['surface'] = df2['stadium'].map(stadium_map).fillna(df2['surface'])

df2.surface.unique()

array(['grass', 'turf'], dtype=object)

In [73]:
# Remove wind direction from weather
df2.weather.str.strip()
df2['weather'] = df2['weather'].str.lower().str.replace('(','').str.replace(')','').str.replace('.','').str.replace("'","").str.replace('n/a','').str.replace('  ', ' ').str.strip()
df2['weather'] = df2['weather'].str.replace(' n ', ' ').str.replace(' nnw ', ' ').str.replace(' wnw ', ' ').str.replace(' nw ', ' ')
df2['weather'] = df2['weather'].str.replace(' w ', ' ').str.replace(' ssw ', ' ').str.replace(' sws ', ' ').str.replace(' sw ', ' ')
df2['weather'] = df2['weather'].str.replace(' s ', ' ').str.replace(' sse ', ' ').str.replace(' ese ', ' ').str.replace(' se ', ' ')
df2['weather'] = df2['weather'].str.replace(' e ', ' ').str.replace(' nne ', ' ').str.replace(' ene ', ' ').str.replace(' ne ', ' ')

df2.weather.nunique()

517

In [74]:
df2.weather.str.strip()

df2['weather'] = np.where(df2['weather'] == 'n/a temp: humidity: wind: mph', np.nan,
                          np.where(df2['weather'] == 'temp: humidity: wind: mph', np.nan,
                                   np.where(df2['weather'] == '', np.nan,
                                            df2['weather'])))

# Impute missing values with previous value within game
df2['weather'] = df2.groupby(['game_id'])['weather'].fillna(method='ffill')

# Map stadiums to new values
weather_map = {'ARI_KC_2022_09_11': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_LA_2022_09_25': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_PHI_2022_10_09': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_NE_2022_12_12': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_NYG_2023_09_17': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_CIN_2023_10_08': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'ARI_ATL_2023_11_12': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_CIN_2022_09_18': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'KC_CIN_2023_01_29': 'mostly cloudy temp: 22° f, humidity: 55%, wind: 10 mph',
               'DAL_TB_2022_09_11':  'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_WAS_2022_10_02': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_DET_2022_10_23': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_NYG_2022_11_24': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_IND_2022_12_04': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_HOU_2022_12_11': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_PHI_2022_12_24': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_NYJ_2023_09_17': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_NE_2023_10_01':  'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_NYG_2023_11_12': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_WAS_2023_11_23': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_SEA_2023_11_30': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'PHI_KC_2023_02_12': 'sunny temp: 77° f, humidity: 9%, wind: 0 mph',
               'DAL_PHI_2023_12_10': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               'DAL_DET_2023_12_30': 'indoors temp: 72° f, humidity: 18%, wind: 0 mph',
               }

# Use weather map values based on game_alt_id
df2['weather'] = np.where(df2['game_alt_id'].isin(weather_map.keys()), df2['game_alt_id'].map(weather_map), df2['weather'])

# Impute missing weather values
# df2['weather'] = np.where(df2['weather'].isnull(), 'indoors temp: 65° f, humidity: 18%, wind: 0 mph', df2['weather'])

# Replace t: with temp:
df2['weather'] = df2['weather'].str.replace('t: ', 'temp: ').str.replace('h: ', 'humidity: ').str.replace('w: ', 'wind: ')

# Print game_alt_id with null values
missing_weather = df2[df2['weather'].isnull()]['game_alt_id'].unique()
print(missing_weather)

[]


In [75]:
# Extract values from weather
df2['game_temp'] = df2['weather'].str.extract(r'temp: (\d+)').astype(float)
df2['game_humidity'] = df2['weather'].str.extract(r'humidity: (\d+)').astype(float)
df2['game_wind'] = df2['weather'].str.extract(r'wind: (\d+)').fillna(0).astype(float)

# Impute with median value
df2['game_temp'] = df2['game_temp'].fillna(df2['game_temp'].median())
df2['game_humidity'] = df2['game_humidity'].fillna(df2['game_humidity'].median())
df2['game_wind'] = df2['game_wind'].fillna(0)

# Code weather conditions, cloudy, sunny, rain, snow, partly cloudy
df2['game_weather'] = np.where(df2['weather'].str.find('controlled climate') >= 0, 'indoor',
                               np.where(df2['weather'].str.find('indoor') >= 0, 'indoor', 
                               np.where(df2['weather'].str[0:7] == 'indoors', 'indoors',
                               np.where(df2['weather'].str[0:18] == 'controlled climate', 'indoors',
                               np.where(df2['weather'].str.find('rain') >= 0, 'rain',
                               np.where(df2['weather'].str.find('drizzle') >= 0, 'rain',
                               np.where(df2['weather'].str.find('snow') >= 0, 'snow',
                               np.where(df2['weather'].str[0:16] == 'scattered clouds', 'cloudy',
                               np.where(df2['weather'].str[0:5] == 'hazey', 'cloudy',
                               np.where(df2['weather'].str.find('overcast') >= 0, 'cloudy',
                               np.where(df2['weather'].str.find('partly') >= 0, 'cloudy',
                               np.where(df2['weather'].str[0:6] == 'cloudy', 'cloudy',
                               np.where(df2['weather'].str.find('sunny') >= 0, 'sunny',
                               np.where(df2['weather'].str.find('clear') >= 0, 'sunny',
                               np.where(df2['weather'].str[0:5] == 'sunny', 'sunny',
                               np.where(df2['weather'].str[0:4] == 'fair', 'sunny',
                               np.where(df2['weather'].str[0:3] == 'fog', 'fog',
                               np.where(df2['weather'].str[0:5] == 'frigid', 'cold',
                               np.where(df2['weather'].str[0:4] == 'cold', 'cold',
                               np.where(df2['weather'].str[0:6] == 'frigid', 'cold',
                               np.where(df2['weather'].str[0:4] == 'sun/', 'sunny',
                               np.where(df2['weather'].str[0:7] == 'drizzle', 'rain',
                               np.where(df2['weather'].str[0:13] == 'thunderstorms', 'thunderstorms',
                               np.where(df2['weather'].str.find('cloudy') >= 0, 'cloudy',
                               'unknown'))))))))))))))))))))))))

df2.game_weather.value_counts(dropna=False, normalize=False)

game_weather
cloudy           25864
sunny            21368
indoor            8590
unknown           5414
rain              4573
snow               480
cold               235
fog                232
thunderstorms      141
Name: count, dtype: int64

In [76]:
# Return the rows with unknown weather
test_df = df2[df2['game_weather'] == 'unknown']
test_df = test_df[['game_id','game_weather','weather']]
test_df.weather.unique()

array(['temp: 74° f, humidity: 91%, wind: 14 mph',
       'temp: 64° f, humidity: 29%, wind: 5 mph',
       'temp: 58° f, humidity: 81%, wind: 13 mph',
       'temp: 62° f, humidity: 35%, wind: 15 mph',
       'temp: 45° f, humidity: 43%, wind: 5 mph',
       'temp: 56° f, humidity: 47%, wind: 8 mph',
       'temp: 66; humidity: 60%; wind: 7mph temp: 66° f, humidity: 60%, wind: northwest 7 mph',
       'temp: 82° f, humidity: 37%, wind: 9 mph',
       'temp: 45° f, humidity: 84%, wind: 15 mph',
       'temp: 61° f, humidity: 60%, wind: 4 mph',
       'temp: 36° f, humidity: 59%, wind: 12 mph',
       'temp: 53° f, humidity: 70%, wind: 15 mph',
       'temp: 51° f, humidity: 47%, wind: 4 mph',
       'temp: 64° f, humidity: 18%, wind: 15 mph',
       'upper 40s to mid 30s by end of game temp: 46° f, humidity: 27%, wind: 8 mph',
       'temp: 53° f, humidity: 71%, wind: 8 mph',
       'temp: 62° f, humidity: 67%, wind: 15 mph',
       'temperatures to remain in 70s throughout the game te

In [77]:
# Impute missing game_weather values with mode
df2['game_weather'] = np.where(df2['game_weather'] =='unknown', df2['game_weather'].mode()[0], df2['game_weather'])

df2.game_weather.value_counts(dropna=False, normalize=False)

game_weather
cloudy           31278
sunny            21368
indoor            8590
rain              4573
snow               480
cold               235
fog                232
thunderstorms      141
Name: count, dtype: int64

In [78]:
test_df.weather.unique()

array(['temp: 74° f, humidity: 91%, wind: 14 mph',
       'temp: 64° f, humidity: 29%, wind: 5 mph',
       'temp: 58° f, humidity: 81%, wind: 13 mph',
       'temp: 62° f, humidity: 35%, wind: 15 mph',
       'temp: 45° f, humidity: 43%, wind: 5 mph',
       'temp: 56° f, humidity: 47%, wind: 8 mph',
       'temp: 66; humidity: 60%; wind: 7mph temp: 66° f, humidity: 60%, wind: northwest 7 mph',
       'temp: 82° f, humidity: 37%, wind: 9 mph',
       'temp: 45° f, humidity: 84%, wind: 15 mph',
       'temp: 61° f, humidity: 60%, wind: 4 mph',
       'temp: 36° f, humidity: 59%, wind: 12 mph',
       'temp: 53° f, humidity: 70%, wind: 15 mph',
       'temp: 51° f, humidity: 47%, wind: 4 mph',
       'temp: 64° f, humidity: 18%, wind: 15 mph',
       'upper 40s to mid 30s by end of game temp: 46° f, humidity: 27%, wind: 8 mph',
       'temp: 53° f, humidity: 71%, wind: 8 mph',
       'temp: 62° f, humidity: 67%, wind: 15 mph',
       'temperatures to remain in 70s throughout the game te

In [79]:
# Print summary stats for game_temp, game_humidity, game_wind
df2[['game_temp','game_humidity','game_wind']].describe()

,game_temp,game_humidity,game_wind
count,66897.000000,66897.000000,66897.000000
mean,60.146045,57.974274,6.819394
std,16.487616,21.444018,5.095072
min,3.000000,0.000000,0.000000
25%,50.000000,44.000000,3.000000
50%,63.000000,60.000000,6.000000
75%,72.000000,73.000000,10.000000
max,93.000000,100.000000,44.000000


In [80]:
df2.offense_formation.value_counts(dropna=False, normalize=False)

offense_formation
SHOTGUN       34667
SINGLEBACK    14113
NaN            5934
EMPTY          4788
I_FORM         3507
PISTOL         2980
JUMBO           572
WILDCAT         336
Name: count, dtype: int64

In [81]:
df2['offense_formation'] = df2['offense_formation'].str.strip().str.lower()

# Impute missing offense_formation with pass formation and run formation
df2['offense_formation'] = np.where((df2['offense_formation'].isnull()) & (df2['shotgun'] == 1),'shotgun', 
                                             np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '6 OL'),'jumbo',
                                                      np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('3 TE') >0),'jumbo',
                                                               np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('0 WR') >0),'jumbo',
                                                                        np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('3 WR') >0), 'shotgun',
                                                                                 np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '2 RB'), 'shotgun',
                                                                                          np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '1 RB'), 'singleback',
                                                                                                   df2['offense_formation'])))))))

# Impute missing offense_formation with mode
df2['offense_formation'] = df2['offense_formation'].fillna(df2['offense_formation'].mode()[0])

df2.offense_formation.value_counts(dropna=False, normalize=False)

offense_formation
shotgun       40598
singleback    14113
empty          4788
i_form         3507
pistol         2980
jumbo           575
wildcat         336
Name: count, dtype: int64

In [82]:
# Offsetting penalties
df2['offsetting_penalties'] = np.where((df2['desc'].str.find('offsetting') >= 0) & (df2['desc'].str.find('no play') >= 0), 1, 0)

# Impute missing yards_gained
df2['yards_gained'] = np.where(df2['offsetting_penalties'] == 1, 0, df2['yards_gained'])

In [83]:
# Code yardline zones
df2['dtg_99to96'] = np.where(df2['yardline_100'] >= 96, 1, 0)
df2['dtg_95to90'] = np.where((df2['yardline_100'] <= 95) & (df2['yardline_100'] >= 90), 1, 0)
df2['dtg_89to75'] = np.where((df2['yardline_100'] <= 89) & (df2['yardline_100'] >= 75), 1, 0)
df2['dtg_74to55'] = np.where((df2['yardline_100'] <= 74) & (df2['yardline_100'] >= 55), 1, 0)
df2['dtg_54to45'] = np.where((df2['yardline_100'] <= 54) & (df2['yardline_100'] >= 45), 1, 0)
df2['dtg_44to35'] = np.where((df2['yardline_100'] <= 44) & (df2['yardline_100'] >= 35), 1, 0)
df2['dtg_34to21'] = np.where((df2['yardline_100'] <= 34) & (df2['yardline_100'] >= 21), 1, 0)
df2['dtg_20to10'] = np.where((df2['yardline_100'] <= 20) & (df2['yardline_100'] >= 10), 1, 0)
df2['dtg_09to00'] = np.where(df2['yardline_100'] <=9, 1, 0)

In [84]:
# Code big gains on previous run plays (>= 15 yards)
df2['big_play_pass'] = np.where((df2['play_type']=='pass') & (df2['yards_gained']>=15) & (df2['turnover']==0), 1, 0)

df2['prev1_big_play_pass'] = df2['big_play_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_big_play_pass'] = df2['big_play_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_big_play_pass'] = df2['big_play_pass'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_big_play_pass_pcnt'] = (df2.groupby(['game_id', 'drive'])['big_play_pass'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert inf values to zero
#df2['drive_big_play_pass_pcnt'] = np.where(df2['drive_big_play_pass_pcnt'] == np.inf, 0, df2['drive_big_play_pass_pcnt'])
#df2['drive_big_play_pass_pcnt'] = np.where(df2['drive_big_play_pass_pcnt'] == -np.inf, 0, df2['drive_big_play_pass_pcnt'])

# Convert values < 0 to zero
df2['drive_big_play_pass_pcnt'] = np.where(df2['drive_big_play_pass_pcnt'] < 0, 0, df2['drive_big_play_pass_pcnt'])

# Drop big play pass
df2.drop(['big_play_pass'], axis=1, inplace=True)

In [85]:
# Code big gains on previous run plays (>= 7 yards)
df2['big_play_run'] = np.where((df2['play_type']=='run') & (df2['yards_gained']>=7) & (df2['turnover']==0), 1, 0)

df2['prev1_big_play_run'] = df2['big_play_run'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_big_play_run'] = df2['big_play_run'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_big_play_run'] = df2['big_play_run'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_big_play_run_pcnt'] = (df2.groupby(['game_id', 'drive'])['big_play_run'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert inf values to zero
df2['drive_big_play_run_pcnt'] = np.where(df2['drive_big_play_run_pcnt'] == np.inf, 0, df2['drive_big_play_run_pcnt'])
df2['drive_big_play_run_pcnt'] = np.where(df2['drive_big_play_run_pcnt'] == -np.inf, 0, df2['drive_big_play_run_pcnt'])

# Convert values < 0 to zero
df2['drive_big_play_run_pcnt'] = np.where(df2['drive_big_play_run_pcnt'] < 0, 0, df2['drive_big_play_run_pcnt'])

# Drop big play run
df2.drop(['big_play_run'], axis=1, inplace=True)

In [86]:
# Code negative run on previous plays
df2['negative_pass'] = np.where((df2['play_type']=='pass') & (df2['yards_gained']<0) & (df2['turnover']==0), 1, 0)

df2['prev1_negative_pass'] = df2['negative_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_negative_pass'] = df2['negative_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_negative_pass'] = df2['negative_pass'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

# Drop negative pass
df2.drop(['negative_pass'], axis=1, inplace=True)

In [87]:
# Code negative run on previous plays
df2['negative_run'] = np.where((df2['play_type']=='run') & (df2['yards_gained'] < 0) & (df2['turnover']==0), 1, 0)

df2['prev1_negative_run'] = df2['negative_run'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_negative_run'] = df2['negative_run'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_negative_run'] = df2['negative_run'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

# Drop negative run
df2.drop(['negative_run'], axis=1, inplace=True)

In [88]:
# Code the play_type on previous plays
df2['play_type_prev1'] = df2['play_type'].shift(1).where(df2['drive'].shift(1) == df2['drive'], np.nan)
df2['play_type_prev2'] = df2['play_type'].shift(2).where(df2['drive'].shift(2) == df2['drive'], np.nan)
df2['play_type_prev3'] = df2['play_type'].shift(3).where(df2['drive'].shift(3) == df2['drive'], np.nan)
df2['play_type_prev4'] = df2['play_type'].shift(4).where(df2['drive'].shift(4) == df2['drive'], np.nan)
df2['play_type_prev5'] = df2['play_type'].shift(5).where(df2['drive'].shift(5) == df2['drive'], np.nan)


# Impute with mode value
df2['play_type_prev1'] = np.where(df2['play_type_prev1'].isnull(), 'none', df2['play_type_prev1'])
df2['play_type_prev2'] = np.where(df2['play_type_prev2'].isnull(), 'none', df2['play_type_prev2'])
df2['play_type_prev3'] = np.where(df2['play_type_prev3'].isnull(), 'none', df2['play_type_prev3'])
df2['play_type_prev4'] = np.where(df2['play_type_prev4'].isnull(), 'none', df2['play_type_prev4'])
df2['play_type_prev5'] = np.where(df2['play_type_prev5'].isnull(), 'none', df2['play_type_prev5'])

df2.play_type_prev1.value_counts(dropna=False)

play_type_prev1
pass    30526
run     24581
none    11790
Name: count, dtype: int64

In [89]:
# Code the play_type on previous plays
df2['penalty_team_prev1'] = df2['penalty_team'].shift(1).where(df2['drive'].shift(1) == df2['drive'], '')
df2['penalty_team_prev2'] = df2['penalty_team'].shift(2).where(df2['drive'].shift(2) == df2['drive'], '')
df2['penalty_team_prev3'] = df2['penalty_team'].shift(3).where(df2['drive'].shift(3) == df2['drive'], '')
df2['penalty_team_prev4'] = df2['penalty_team'].shift(4).where(df2['drive'].shift(4) == df2['drive'], '')
df2['penalty_team_prev5'] = df2['penalty_team'].shift(5).where(df2['drive'].shift(5) == df2['drive'], '')

##########
df2['prev1_play_off_penalty'] = np.where(df2['penalty_team_prev1'] == df2['posteam'], 1, 0)
df2['prev1_play_def_penalty'] = np.where(df2['penalty_team_prev1'] == df2['defteam'], 1, 0)

##########
df2['prev2_play_off_penalty'] = np.where(df2['penalty_team_prev2'] == df2['posteam'], 1, 0)
df2['prev2_play_def_penalty'] = np.where(df2['penalty_team_prev2'] == df2['defteam'], 1, 0)

##########
df2['prev3_play_off_penalty'] = np.where(df2['penalty_team_prev3'] == df2['posteam'], 1, 0)
df2['prev3_play_def_penalty'] = np.where(df2['penalty_team_prev3'] == df2['defteam'], 1, 0)

##########
df2['prev4_play_off_penalty'] = np.where(df2['penalty_team_prev4'] == df2['posteam'], 1, 0)
df2['prev4_play_def_penalty'] = np.where(df2['penalty_team_prev4'] == df2['defteam'], 1, 0)

##########
df2['prev5_play_off_penalty'] = np.where(df2['penalty_team_prev5'] == df2['posteam'], 1, 0)
df2['prev5_play_def_penalty'] = np.where(df2['penalty_team_prev5'] == df2['defteam'], 1, 0)

#########
df2.drop(['penalty_team','penalty_team_prev1','penalty_team_prev2','penalty_team_prev3','penalty_team_prev4','penalty_team_prev5'], axis=1, inplace=True)

In [90]:
# Code the play_type_detail on previous plays
df2['play_type_detail_prev1'] = df2['play_type_detail'].shift(1).where(df2['drive'].shift(1) == df2['drive'], '')
df2['play_type_detail_prev2'] = df2['play_type_detail'].shift(2).where(df2['drive'].shift(2) == df2['drive'], '')
df2['play_type_detail_prev3'] = df2['play_type_detail'].shift(3).where(df2['drive'].shift(3) == df2['drive'], '')

df2['prev1_play_run_outside'] = np.where(df2['play_type_detail_prev1'] == 'run_outside', 1, 0)
df2['prev1_play_run_inside'] = np.where(df2['play_type_detail_prev1'] == 'run_inside', 1, 0)
df2['prev1_play_pass_deep'] = np.where(df2['play_type_detail_prev1'] == 'pass_deep', 1, 0)
df2['prev1_play_pass_short'] = np.where(df2['play_type_detail_prev1'] == 'pass_short', 1, 0)

df2['prev2_play_run_outside'] = np.where(df2['play_type_detail_prev2'] == 'run_outside', 1, 0)
df2['prev2_play_run_inside'] = np.where(df2['play_type_detail_prev2'] == 'run_inside', 1, 0)
df2['prev2_play_pass_deep'] = np.where(df2['play_type_detail_prev2'] == 'pass_deep', 1, 0)
df2['prev2_play_pass_short'] = np.where(df2['play_type_detail_prev2'] == 'pass_short', 1, 0)

df2['prev3_play_run_outside'] = np.where(df2['play_type_detail_prev3'] == 'run_outside', 1, 0)
df2['prev3_play_run_inside'] = np.where(df2['play_type_detail_prev3'] == 'run_inside', 1, 0)
df2['prev3_play_pass_deep'] = np.where(df2['play_type_detail_prev3'] == 'pass_deep', 1, 0)
df2['prev3_play_pass_short'] = np.where(df2['play_type_detail_prev3'] == 'pass_short', 1, 0)

df2.drop(['play_type_detail_prev1','play_type_detail_prev2','play_type_detail_prev3'], axis=1, inplace=True)

In [91]:
# Code incomplete passes on previous plays
df2['prev1_incomplete_pass'] = df2['incomplete_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_incomplete_pass'] = df2['incomplete_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_incomplete_pass'] = df2['incomplete_pass'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_incomplete_pass'] = df2['incomplete_pass'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)

df2['prev1_incomplete_pass'].fillna(0, inplace = True)
df2['prev2_incomplete_pass'].fillna(0, inplace = True)
df2['prev3_incomplete_pass'].fillna(0, inplace = True)
df2['prev4_incomplete_pass'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_incomplete_pass_pcnt'] = (df2.groupby(['game_id', 'drive'])['incomplete_pass'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert values < 0 to zero
df2['drive_incomplete_pass_pcnt'] = np.where(df2['drive_incomplete_pass_pcnt'] < 0, 0, df2['drive_incomplete_pass_pcnt'])

# Convert Inf values to zero
df2['drive_incomplete_pass_pcnt'] = np.where(df2['drive_incomplete_pass_pcnt'] == np.inf, 0, df2['drive_incomplete_pass_pcnt'])
df2['drive_incomplete_pass_pcnt'] = np.where(df2['drive_incomplete_pass_pcnt'] == -np.inf, 0, df2['drive_incomplete_pass_pcnt'])

In [92]:
# Code yards on previous plays
df2['prev1_yards_gained'] = df2['yards_gained'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_yards_gained'] = df2['yards_gained'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_yards_gained'] = df2['yards_gained'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_yards_gained'] = df2['yards_gained'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_yards_gained'] = df2['yards_gained'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

df2['prev1_yards_gained'].fillna(0, inplace = True)
df2['prev2_yards_gained'].fillna(0, inplace = True)
df2['prev3_yards_gained'].fillna(0, inplace = True)
df2['prev4_yards_gained'].fillna(0, inplace = True)
df2['prev5_yards_gained'].fillna(0, inplace = True)

In [93]:
# Code win probability of previous plays
df2['prev1_wpa'] = df2['wpa'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_wpa'] = df2['wpa'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_wpa'] = df2['wpa'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_wpa'] = df2['wpa'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_wpa'] = df2['wpa'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

df2['prev1_wpa'].fillna(0, inplace = True)
df2['prev2_wpa'].fillna(0, inplace = True)
df2['prev3_wpa'].fillna(0, inplace = True)
df2['prev4_wpa'].fillna(0, inplace = True)
df2['prev5_wpa'].fillna(0, inplace = True)

In [94]:
# Code shotgun formation of previous plays
df2['prev1_shotgun'] = df2['shotgun'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_shotgun'] = df2['shotgun'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_shotgun'] = df2['shotgun'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_shotgun'] = df2['shotgun'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_shotgun'] = df2['shotgun'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

df2['prev1_shotgun'].fillna(0, inplace = True)
df2['prev2_shotgun'].fillna(0, inplace = True)
df2['prev3_shotgun'].fillna(0, inplace = True)
df2['prev4_shotgun'].fillna(0, inplace = True)
df2['prev5_shotgun'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_shotgun_pcnt'] = (df2.groupby(['game_id', 'drive'])['shotgun'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert inf values to zero
df2['drive_shotgun_pcnt'] = np.where(df2['drive_shotgun_pcnt'] == np.inf, 0, df2['drive_shotgun_pcnt'])

# Convert values < 0 to zero
df2['drive_shotgun_pcnt'] = np.where(df2['drive_shotgun_pcnt'] < 0, 0, df2['drive_shotgun_pcnt'])

In [95]:
# Code qb_hit on previous plays
df2['prev1_qb_hit'] = df2['qb_hit'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_qb_hit'] = df2['qb_hit'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_qb_hit'] = df2['qb_hit'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_qb_hit'] = df2['qb_hit'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_qb_hit'] = df2['qb_hit'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

df2['prev1_qb_hit'].fillna(0, inplace = True)
df2['prev2_qb_hit'].fillna(0, inplace = True)
df2['prev3_qb_hit'].fillna(0, inplace = True)
df2['prev4_qb_hit'].fillna(0, inplace = True)
df2['prev5_qb_hit'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_qb_hit_pcnt'] = (df2.groupby(['game_id', 'drive'])['qb_hit'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert values < 0 to zero
df2['drive_qb_hit_pcnt'] = np.where(df2['drive_qb_hit_pcnt'] < 0, 0, df2['drive_qb_hit_pcnt'])

# Convert Inf values to zero
df2['drive_qb_hit_pcnt'] = np.where(df2['drive_qb_hit_pcnt'] == np.inf, 0, df2['drive_qb_hit_pcnt'])
df2['drive_qb_hit_pcnt'] = np.where(df2['drive_qb_hit_pcnt'] == -np.inf, 0, df2['drive_qb_hit_pcnt'])

# Drop qb_hit
df2.drop(['qb_hit'], axis=1, inplace=True)

In [96]:
# Code no huddle of previous plays
df2['prev1_no_huddle'] = df2['no_huddle'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_no_huddle'] = df2['no_huddle'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_no_huddle'] = df2['no_huddle'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_no_huddle'] = df2['no_huddle'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_no_huddle'] = df2['no_huddle'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

df2['prev1_no_huddle'].fillna(0, inplace = True)
df2['prev2_no_huddle'].fillna(0, inplace = True)
df2['prev3_no_huddle'].fillna(0, inplace = True)
df2['prev4_no_huddle'].fillna(0, inplace = True)
df2['prev5_no_huddle'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_no_huddle_pcnt'] = (df2.groupby(['game_id', 'drive'])['no_huddle'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert values < 0 to zero
df2['drive_no_huddle_pcnt'] = np.where(df2['drive_no_huddle_pcnt'] < 0, 0, df2['drive_no_huddle_pcnt'])

# Convert Inf values to zero
df2['drive_no_huddle_pcnt'] = np.where(df2['drive_no_huddle_pcnt'] == np.inf, 0, df2['drive_no_huddle_pcnt'])
df2['drive_no_huddle_pcnt'] = np.where(df2['drive_no_huddle_pcnt'] == -np.inf, 0, df2['drive_no_huddle_pcnt'])


In [97]:
# Code first down achieved on a previous play
df2['prev1_first_down_pass'] = df2['first_down_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_first_down_pass'] = df2['first_down_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_first_down_pass'] = df2['first_down_pass'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

df2['prev1_first_down_pass'].fillna(0, inplace = True)
df2['prev2_first_down_pass'].fillna(0, inplace = True)
df2['prev3_first_down_pass'].fillna(0, inplace = True)

# Drop first_down_pass
df2.drop(['first_down_pass'], axis=1, inplace=True)

In [98]:
# Code first down achieved on a previous play
df2['prev1_first_down_run'] = df2['first_down_rush'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_first_down_run'] = df2['first_down_rush'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_first_down_run'] = df2['first_down_rush'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

df2['prev1_first_down_run'].fillna(0, inplace = True)
df2['prev2_first_down_run'].fillna(0, inplace = True)
df2['prev3_first_down_run'].fillna(0, inplace = True)

# Drop first_down_rush
df2.drop(['first_down_rush'], axis=1, inplace=True)

In [99]:
# Code play efficiency (1st down >= 4, 2nd down half the distance, 3rd and 4th down = first down)
df2['effct_play'] = np.where((df2['down']==1) & (df2['yards_gained'] >= 4) & (df2['turnover'] == 0), 1,
                             np.where((df2['down']==2) & (df2['yards_gained'] >= (df2['ydstogo']/2)) & (df2['turnover'] == 0), 1,
                                      np.where((df2['down']==3) & (df2['yards_gained'] >= df2['ydstogo']) & (df2['turnover'] == 0), 1,
                                               np.where((df2['down']==4) & (df2['yards_gained'] >= df2['ydstogo']) & (df2['turnover'] == 0), 1, 0))))

df2['prev1_effct_play'] = df2['effct_play'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_effct_play'] = df2['effct_play'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_effct_play'] = df2['effct_play'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)
df2['prev4_effct_play'] = df2['effct_play'].shift(4).where(df2['drive'].shift(4) == df2['drive'], 0)
df2['prev5_effct_play'] = df2['effct_play'].shift(5).where(df2['drive'].shift(5) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_effct_play_pcnt'] = (df2.groupby(['game_id', 'drive'])['effct_play'].transform(lambda x: x.rolling(window=rolling_series_window, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

# Convert Inf values to zero
df2['drive_effct_play_pcnt'] = np.where(df2['drive_effct_play_pcnt'] == np.inf, 0, df2['drive_effct_play_pcnt'])
df2['drive_effct_play_pcnt'] = np.where(df2['drive_effct_play_pcnt'] == -np.inf, 0, df2['drive_effct_play_pcnt'])

# Convert values < 0 to zero
df2['drive_effct_play_pcnt'] = np.where(df2['drive_effct_play_pcnt'] < 0, 0, df2['drive_effct_play_pcnt'])

# Drop effct_play
df2.drop(['effct_play'], axis=1, inplace=True)

In [100]:
# Code the offensive personnel on previous plays
df2['offense_personnel_prev1'] = df2['offense_personnel'].shift(1).where(df2['drive'].shift(1) == df2['drive'], np.nan)
df2['offense_personnel_prev2'] = df2['offense_personnel'].shift(2).where(df2['drive'].shift(2) == df2['drive'], np.nan)
df2['offense_personnel_prev3'] = df2['offense_personnel'].shift(3).where(df2['drive'].shift(3) == df2['drive'], np.nan)

df2['offense_personnel_prev1'] = np.where(df2['offense_personnel_prev1'].isnull(), 'none', df2['offense_personnel_prev1'])
df2['offense_personnel_prev2'] = np.where(df2['offense_personnel_prev2'].isnull(), 'none', df2['offense_personnel_prev2'])
df2['offense_personnel_prev3'] = np.where(df2['offense_personnel_prev3'].isnull(), 'none', df2['offense_personnel_prev3'])

# Code the offensive personnel on previous plays
df2['defense_personnel_prev1'] = df2['defense_personnel'].shift(1).where(df2['drive'].shift(1) == df2['drive'], np.nan)
df2['defense_personnel_prev2'] = df2['defense_personnel'].shift(2).where(df2['drive'].shift(2) == df2['drive'], np.nan)
df2['defense_personnel_prev3'] = df2['defense_personnel'].shift(3).where(df2['drive'].shift(3) == df2['drive'], np.nan)

df2['defense_personnel_prev1'] = np.where(df2['defense_personnel_prev1'].isnull(), 'none', df2['defense_personnel_prev1'])
df2['defense_personnel_prev2'] = np.where(df2['defense_personnel_prev2'].isnull(), 'none', df2['defense_personnel_prev2'])
df2['defense_personnel_prev3'] = np.where(df2['defense_personnel_prev3'].isnull(), 'none', df2['defense_personnel_prev3'])


In [101]:
# Code downs_remaining
df2['remaining_downs'] = np.where(df2['down'] == 1, 3, 
                                  np.where(df2['down'] == 2, 2,
                                           np.where((df2['down'] == 3) & (df2['yardline_100'] <= 55) & (df2['yardline_100'] >= 33) & (df2['ydstogo'] <= 8), 2,
                                                    np.where(df2['down'] == 3, 1,
                                                             np.where(df2['down'] == 4, 1, 1)))))

# Code yards_per_down remaining
df2['remaining_yards_per_down'] = df2['ydstogo'] / df2['remaining_downs']

In [102]:
df2['game_half'] = np.where(df2['game_half'] == 'Half1', 1, np.where(df2['game_half'] == 'Half2', 2, 3))

In [103]:
df2['drive_start_transition'] = df2['drive_start_transition'].str.strip().str.replace(',','').replace('_','').str.strip().str.upper()

# Create mapping between drive start transition and drive start category
drive_start_map = {'INTERCEPTION': 'sudden_change',
                   'FUMBLE': 'sudden_change',
                   'MUFFED_PUNT': 'sudden_change',
                   'ONSIDE_KICK': 'sudden_change',
                   'BLOCKED_FG': 'sudden_change',
                   'BLOCKED_PUNT': 'sudden_change',
                   'BLOCKED_FG_DOWNS': 'sudden_change',
                   'BLOCKED_FG, DOWNS': 'sudden_change',
                   'BLOCKED_PUNT_DOWNS': 'sudden_change',
                   'MUFFED_KICKOFF': 'sudden_change',
                   'OWN_KICKOFF': 'sudden_change',
                   'DOWNS': 'transfer_poss',
                   'MISSED_FG': 'transfer_poss',
                   'KICKOFF': 'transfer_poss',
                   'PUNT': 'transfer_poss',
                   }

df2['drive_start'] = df2['drive_start_transition'].map(drive_start_map).fillna(df2['drive_start_transition'])
df2['drive_start'].fillna('NULL', inplace=True)

df2.drive_start.value_counts(dropna=False)

drive_start
transfer_poss    60868
sudden_change     6029
Name: count, dtype: int64

In [104]:
df2['two_min_warning'] = np.where(df2['half_seconds_remaining'] <= 120, 1, 0)

In [105]:
df2['ep_half_sec_ratio'] = (df2['ep'] / (df2['half_seconds_remaining'] + 1 ))
df2['ep_game_sec_ratio'] = (df2['ep'] / (df2['game_seconds_remaining'] + 1 ))

# Convert Inf values to zero
df2['ep_half_sec_ratio'] = np.where(df2['ep_half_sec_ratio'] == np.inf, 0, df2['ep_half_sec_ratio'])
df2['ep_game_sec_ratio'] = np.where(df2['ep_game_sec_ratio'] == np.inf, 0, df2['ep_game_sec_ratio'])

# Convert Inf values to zero
df2['ep_half_sec_ratio'] = np.where(df2['ep_half_sec_ratio'] == -np.inf, 0, df2['ep_half_sec_ratio'])
df2['ep_game_sec_ratio'] = np.where(df2['ep_game_sec_ratio'] == -np.inf, 0, df2['ep_game_sec_ratio'])

In [201]:
# Duplicate dataframe
df3 = df2.copy()

df3.shape

(66897, 443)

In [202]:
# Rename target classes
df3['play_type_detail'] = np.where(df3['play_type_detail'] == 'pass_short', 'short',
                                   np.where(df3['play_type_detail'] == 'pass_deep', 'deep',
                                            np.where(df3['play_type_detail'] == 'run_inside', 'inside',
                                                     np.where(df3['play_type_detail'] == 'run_outside', 'outside',
                                                              df3['play_type_detail']))))

## IMPUTE MISSING VALUES

In [203]:
# Impute missing offense_personnel using the previous play within drive
df3['offense_personnel'] = df3['offense_personnel'].fillna(df3.groupby(['game_id', 'drive'])['offense_personnel'].transform('last'))

df3.offense_personnel.isnull().sum()

0

In [204]:
df3['defense_personnel'] = df3['defense_personnel'].fillna(df3.groupby(['game_id', 'drive'])['defense_personnel'].transform('last'))

df3.defense_personnel.isnull().sum()

0

In [205]:
# Impute defenders_in_box
df3['defenders_in_box'] = df3['defenders_in_box'].fillna(df3.groupby(['defteam'])['defenders_in_box'].transform('mean'))

# Print missing value counts
df3['defenders_in_box'].isnull().sum()

0

## FEATURE ENGINEERING

In [206]:
# Binary code for side of field = posteam
df3['posteam_side'] = np.where(df3['posteam'] == df3['side_of_field'], 1, 0)

# Drop side_of_field
df3.drop(['side_of_field'], axis=1, inplace=True)

In [207]:
# Combine half_seconds_remaining and score_differential_norm
df3['half_seconds_div_score_diff'] = (df3['half_seconds_remaining'] + 1) / df3['score_differential_norm']
df3['half_seconds_prod_score_diff'] = (df3['half_seconds_remaining'] + 1) * df3['score_differential_norm']

# Convert Inf values to zero
df3['half_seconds_div_score_diff'] = np.where(df3['half_seconds_div_score_diff'] == np.inf, 0, df3['half_seconds_div_score_diff'])
df3['half_seconds_prod_score_diff'] = np.where(df3['half_seconds_prod_score_diff'] == np.inf, 0, df3['half_seconds_prod_score_diff'])

# Convert Inf values to zero
df3['half_seconds_div_score_diff'] = np.where(df3['half_seconds_div_score_diff'] == -np.inf, 0, df3['half_seconds_div_score_diff'])
df3['half_seconds_prod_score_diff'] = np.where(df3['half_seconds_prod_score_diff'] == -np.inf, 0, df3['half_seconds_prod_score_diff'])

In [208]:
# Combine wind and temperature
df3['game_wind_div_game_temp'] = df3['game_wind'] / df3['game_temp']
df3['game_wind_prod_game_temp'] = df3['game_wind'] * df3['game_temp']

# Combine temperature and humidity
df3['game_temp_div_game_humidity'] = df3['game_temp'] / df3['game_humidity']
df3['game_temp_prod_game_humidity'] = df3['game_temp'] * df3['game_humidity']

# Convert inf values to zero
df3['game_wind_div_game_temp'] = np.where(df3['game_wind_div_game_temp'] == np.inf, 0, df3['game_wind_div_game_temp'])
df3['game_wind_prod_game_temp'] = np.where(df3['game_wind_prod_game_temp'] == np.inf, 0, df3['game_wind_prod_game_temp'])
df3['game_temp_div_game_humidity'] = np.where(df3['game_temp_div_game_humidity'] == np.inf, 0, df3['game_temp_div_game_humidity'])
df3['game_temp_prod_game_humidity'] = np.where(df3['game_temp_prod_game_humidity'] == np.inf, 0, df3['game_temp_prod_game_humidity'])

# Convert -inf values to zero
df3['game_wind_div_game_temp'] = np.where(df3['game_wind_div_game_temp'] == -np.inf, 0, df3['game_wind_div_game_temp'])
df3['game_wind_prod_game_temp'] = np.where(df3['game_wind_prod_game_temp'] == -np.inf, 0, df3['game_wind_prod_game_temp'])
df3['game_temp_div_game_humidity'] = np.where(df3['game_temp_div_game_humidity'] == -np.inf, 0, df3['game_temp_div_game_humidity'])
df3['game_temp_prod_game_humidity'] = np.where(df3['game_temp_prod_game_humidity'] == -np.inf, 0, df3['game_temp_prod_game_humidity'])


In [209]:
# Calculate average run ratio per game
rro_df = df3.groupby(['posteam','game_id','qtr','down'])['rush_attempt'].apply(lambda x : x.astype(float).mean()).reset_index()

# Sort dataframe
rro_df = rro_df.sort_values(by=['posteam','qtr','down'], ascending=True)

# Calculate rolling average per quarter
rro_df['run_ratio_off_priors'] = rro_df.groupby(['posteam','qtr','down'])['rush_attempt'].transform(lambda x: x.rolling(window=rolling_play_quarter_window,
                                                                                                                        min_periods=1,
                                                                                                                        closed='left',
                                                                                                                        center=False).mean())

rro_df = rro_df.sort_values(by=['posteam','qtr','down'], ascending=True)

rro_df.drop(['rush_attempt'], axis=1, inplace=True)

# Impute missing values by the league average per qtr/down
rro_df['run_ratio_off_priors'] = rro_df['run_ratio_off_priors'].fillna(rro_df.groupby(['qtr','down'])['run_ratio_off_priors'].transform('mean'))

# Merge msa_df with df
df3 = df3.merge(rro_df, how='left', left_on=['posteam','game_id','qtr','down'], right_on=['posteam','game_id','qtr','down'])

# Impute values
df3['run_ratio_off_priors'] = df3['run_ratio_off_priors'].fillna(df3.groupby(['game_id', 'posteam'])['run_ratio_off_priors'].transform('last'))

df3['run_ratio_off_priors'] = np.where(df3['run_ratio_off_priors'] < 0, 0, df3['run_ratio_off_priors'])

In [210]:
# Calculate average run ratio per game
rrd_df = df3.groupby(['defteam','game_id','qtr','down'])['rush_attempt'].apply(lambda x : x.astype(float).mean()).reset_index()

# Sort dataframe
rrd_df = rrd_df.sort_values(by=['defteam','qtr','down'], ascending=True)

# Calculate rolling average per quarter
rrd_df['run_ratio_def_priors'] = rrd_df.groupby(['defteam','qtr','down'])['rush_attempt'].transform(lambda x: x.rolling(window=rolling_play_quarter_window,
                                                                                                                        min_periods=1,
                                                                                                                        closed='left',
                                                                                                                        center=False).mean())

rrd_df = rrd_df.sort_values(by=['defteam','qtr','down'], ascending=True)

rrd_df.drop(['rush_attempt'], axis=1, inplace=True)

# Impute missing values by the league average per qtr/down
rrd_df['run_ratio_def_priors'] = rrd_df['run_ratio_def_priors'].fillna(rrd_df.groupby(['qtr','down'])['run_ratio_def_priors'].transform('mean'))

# Merge msa_df with df
df3 = df3.merge(rrd_df, how='left', left_on=['defteam','game_id','qtr','down'], right_on=['defteam','game_id','qtr','down'])

# Impute defenders_in_box
df3['run_ratio_def_priors'] = df3['run_ratio_def_priors'].fillna(df3.groupby(['game_id', 'posteam'])['run_ratio_def_priors'].transform('last'))

In [211]:
# Create posteam plus season and defteam plus season
df3['posteam_season'] = df3['posteam'].str.lower().str.strip() + '_' + df3['season'].astype(str)
df3['defteam_season'] = df3['defteam'].str.lower().str.strip() + '_' + df3['season'].astype(str)

## FUTURE FEATURES

## PLAY PREDICTION FILTERS

In [212]:
df3.shape

(66897, 453)

In [213]:
df3 = df3.loc[~df3['desc'].str.contains('punt formation')]
df3 = df3.loc[~df3['desc'].str.contains('field goal formation')]

df3.shape

(66895, 453)

In [214]:
df3 = df3.loc[df3['drive_start'] != 'NULL']

df3.shape

(66895, 453)

In [215]:
df3 = df3.loc[(df3['play_type'] == 'pass') | (df3['play_type'] == 'run')]

df3.shape

(66895, 453)

In [216]:
df3 = df3[~df3['play_type_detail'].isin(['qb_scramble','qb_spike','qb_kneel'])]

df3.shape

(66895, 453)

In [217]:
# Code and filter "broken plays" (plays with a fumble prior to the scripted play could be executed0
df3['play_type'] = np.where((df3['play_type_detail'] == 'run') & (df3['desc'].str.find('fumbles')>= 0) & (df3['desc'].str.find(', and recovers')>= 0), 'broken_play', 
                            np.where((df3['play_type_detail'] == 'run') & (df3['desc'].str.find('fumbles')>= 0) & (df3['desc'].str.find('ball out of bounds')>= 0), 'broken_play', 
                                     np.where((df3['play_type_detail'] == 'run') & (df3['desc'].str.find('fumbles')>= 0) & (df3['desc'].str.find(', recovered by')>= 0), 'broken_play',
                                     df3['play_type'])))

df3 = df3[df3['play_type'] != 'broken_play']

df3.shape

(66871, 453)

In [218]:
counts = df3.play_type_detail.value_counts(normalize=False)
counts

play_type_detail
short        26244
inside       13950
outside      12984
deep         10926
pass_sack     2765
pass             2
Name: count, dtype: int64

In [219]:
# Return rows where play_type_detail is not in the top 5
test_df = df3.loc[df3['play_type_detail'].isin(['run','pass'])]
test_df

,play_id,game_id,old_game_id,game_alt_id,home_team,away_team,season_type,week,posteam,posteam_type,...,half_seconds_div_score_diff,half_seconds_prod_score_diff,game_wind_div_game_temp,game_wind_prod_game_temp,game_temp_div_game_humidity,game_temp_prod_game_humidity,run_ratio_off_priors,run_ratio_def_priors,posteam_season,defteam_season
42057,1644,2022_04_MIN_NO,2022100200,NO_MIN_2022_10_02,NO,MIN,REG,4,MIN,away,...,230.928571,64.452830,0.09375,384.0,0.810127,5056.0,0.276353,0.583333,min_2022,no_2022
58917,3990,2022_07_KC_SF,2022102310,SF_KC_2022_10_23,SF,KC,REG,7,SF,home,...,1044.857143,113.915094,0.15625,640.0,1.882353,2176.0,0.402778,0.191667,sf_2022,kc_2022


## DROP COLUMNS

In [220]:
# Print a list of columns with only one value and then drop them
single_value_columns = df3.loc[:,df3.nunique() == 1].columns
df3 = df3.loc[:,df3.nunique() > 1]

print('Single value columns:', single_value_columns)
print('Remaining columns:', df3.shape[1])

Single value columns: Index(['qb_scramble', 'offsetting_penalties', 'drive_big_play_pass_pcnt',
       'drive_big_play_run_pcnt', 'drive_incomplete_pass_pcnt',
       'drive_shotgun_pcnt', 'drive_qb_hit_pcnt', 'drive_no_huddle_pcnt',
       'drive_effct_play_pcnt'],
      dtype='object')
Remaining columns: 444


In [221]:
# Drop unused features
df3.drop(['old_game_id','game_alt_id','season_type','game_date','drive','time','yrdln','ydsnet','play','desc','qb_dropback','air_yards','yards_after_catch','timeout','timeout_team','td_team','td_player_name','td_player_id','total_home_score','total_away_score','posteam_score_post','defteam_score_post','score_differential_post','opp_fg_prob','opp_safety_prob','opp_td_prob','total_home_epa','total_away_epa','total_home_rush_epa','total_away_rush_epa','total_home_pass_epa','total_away_pass_epa','air_epa','yac_epa','comp_air_epa','comp_yac_epa','total_home_comp_air_epa','total_away_comp_air_epa','total_home_comp_yac_epa','total_away_comp_yac_epa','total_home_raw_air_epa','total_away_raw_air_epa','total_home_raw_yac_epa','total_away_raw_yac_epa','def_wp','home_wp','away_wp','wpa','vegas_wpa','vegas_home_wpa','home_wp_post','away_wp_post','vegas_wp','vegas_home_wp','total_home_rush_wpa','total_away_rush_wpa','total_home_pass_wpa','total_away_pass_wpa','air_wpa','yac_wpa','comp_air_wpa','comp_yac_wpa','total_home_comp_air_wpa','total_away_comp_air_wpa','total_home_comp_yac_wpa','total_away_comp_yac_wpa','total_home_raw_air_wpa','total_away_raw_air_wpa','total_home_raw_yac_wpa','total_away_raw_yac_wpa','first_down_penalty','third_down_converted','third_down_failed','fourth_down_converted','fourth_down_failed','incomplete_pass','touchback','interception','fumble_forced','fumble_not_forced','fumble_out_of_bounds','solo_tackle','sack','safety','penalty','tackled_for_loss','fumble_lost','touchdown','pass_touchdown','rush_touchdown','return_touchdown','fumble','complete_pass','assist_tackle','lateral_reception','lateral_rush','lateral_return','lateral_recovery','passer_player_id','passer_player_name','passing_yards','receiver_player_id','receiver_player_name','receiving_yards','rusher_player_id','rusher_player_name','rushing_yards','lateral_receiver_player_id','lateral_receiver_player_name','lateral_receiving_yards','lateral_rusher_player_id','lateral_rusher_player_name','lateral_rushing_yards','interception_player_id','interception_player_name','lateral_interception_player_id','lateral_interception_player_name','tackle_for_loss_1_player_id','tackle_for_loss_1_player_name','qb_hit_1_player_id','qb_hit_1_player_name','qb_hit_2_player_id','qb_hit_2_player_name','sack_player_id','sack_player_name', 'half_sack_1_player_id', 'half_sack_1_player_name','half_sack_2_player_id', 'half_sack_2_player_name','forced_fumble_player_1_team','forced_fumble_player_1_player_id','forced_fumble_player_1_player_name','forced_fumble_player_2_team','forced_fumble_player_2_player_id','forced_fumble_player_2_player_name','solo_tackle_1_team','solo_tackle_2_team','solo_tackle_1_player_id','solo_tackle_2_player_id','solo_tackle_1_player_name','solo_tackle_2_player_name','assist_tackle_1_player_id','assist_tackle_1_player_name','assist_tackle_1_team','assist_tackle_2_player_id','assist_tackle_2_player_name','assist_tackle_2_team','tackle_with_assist','tackle_with_assist_1_player_id','tackle_with_assist_1_player_name','tackle_with_assist_1_team','pass_defense_1_player_id','pass_defense_1_player_name','pass_defense_2_player_id','pass_defense_2_player_name','fumbled_1_team','fumbled_1_player_id','fumbled_1_player_name','fumbled_2_player_id','fumbled_2_player_name','fumbled_2_team','fumble_recovery_1_team','fumble_recovery_1_yards','fumble_recovery_1_player_id','fumble_recovery_1_player_name','fumble_recovery_2_team','fumble_recovery_2_yards','fumble_recovery_2_player_id','fumble_recovery_2_player_name','return_team','return_yards','penalty_player_id','penalty_player_name','penalty_yards','replay_or_challenge','replay_or_challenge_result','penalty_type','safety_player_name','safety_player_id','series','series_success','series_result','start_time','time_of_day','stadium','weather','nfl_api_id','play_clock','play_type_nfl','end_clock_time','end_yard_line','fixed_drive','fixed_drive_result','drive_real_start_time','drive_time_of_possession','drive_first_downs','drive_inside20','drive_ended_with_score','drive_quarter_start','drive_quarter_end','drive_yards_penalized','drive_end_transition','drive_start_transition','drive_game_clock_start','drive_game_clock_end','drive_start_yard_line','drive_end_yard_line','drive_play_id_started','drive_play_id_ended','away_score','home_score','location','result','total','home_coach','away_coach','stadium_id','game_stadium','success','passer','passer_jersey_number','rusher','rusher_jersey_number','receiver','receiver_jersey_number','pass','rush','first_down','passer_id','rusher_id','receiver_id','name','jersey_number','id','fantasy_player_name','fantasy_player_id','fantasy','fantasy_id','out_of_bounds','home_opening_kickoff','qb_epa','xyac_epa','xyac_mean_yardage','xyac_median_yardage','xyac_success','xyac_fd','xpass','pass_oe','cp','cpoe','pass_length','pass_location','run_location','run_gap','pass_attempt','game_id','home_team','away_team','sp','yards_gained','home_timeouts_remaining','away_timeouts_remaining','order_sequence','play_id','epa','turnover','rush_attempt','nflverse_game_id','players_on_play','offense_players','defense_players','number_of_pass_rushers','season','temp','wind','possession_team','time_to_throw','was_pressure','route','defense_coverage_type','defense_man_zone_type','ngs_air_yards','n_offense','n_defense','drive_play_count','score_differential','play_type_detail','td_prob','fg_prob','game_temp','game_humidity','game_wind','down','posteam_side','half_seconds_prod_score_diff'], axis=1, inplace=True)

df3.shape

(66871, 149)

In [222]:
# Create a list of columns with any inf values
inf_columns = df3.columns[df3.isin([np.inf, -np.inf]).any()]

# Drop inf_columns from dataframe
df3.drop(inf_columns, axis=1, inplace=True)
print(inf_columns)

Index([], dtype='object')


In [223]:
# Drop columns with missing values
null_columns = df3.columns[df3.isnull().any()]

df3.dropna(axis=0, inplace=True)
print('Dropping the following columns with missing values:')
print(null_columns)

Dropping the following columns with missing values:
Index([], dtype='object')


In [236]:
df4 = df3.copy()

# Print list of columns
list(df4.columns)

['week',
 'posteam',
 'posteam_type',
 'defteam',
 'yardline_100',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'qtr',
 'goal_to_go',
 'ydstogo',
 'play_type',
 'shotgun',
 'no_huddle',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'posteam_score',
 'defteam_score',
 'no_score_prob',
 'safety_prob',
 'ep',
 'wp',
 'spread_line',
 'total_line',
 'div_game',
 'roof',
 'surface',
 'offense_formation',
 'offense_personnel',
 'defenders_in_box',
 'defense_personnel',
 'report_eligible',
 'wr_to_db_ratio',
 'ol_to_dl_ratio',
 'intentional_grounding',
 'play_sequence_game',
 'play_sequence_series',
 'score_differential_norm',
 'game_weather',
 'dtg_99to96',
 'dtg_95to90',
 'dtg_89to75',
 'dtg_74to55',
 'dtg_54to45',
 'dtg_44to35',
 'dtg_34to21',
 'dtg_20to10',
 'dtg_09to00',
 'prev1_big_play_pass',
 'prev2_big_play_pass',
 'prev3_big_play_pass',
 'prev1_big_play_run',
 'prev2_big_play_run',
 'prev3_big_play_run',
 'prev1_n

In [237]:
# Convert target variable to binary
df4['play_type'] = np.where(df4['play_type'] == 'pass', 1, 0)

# Print value counts
df4['play_type'].value_counts(dropna=False, normalize=True)

play_type
1    0.597225
0    0.402775
Name: proportion, dtype: float64

## FEATURE SIGNIFICANCE + MULTICOLLINEARITY

In [238]:
# Create a list of categorical variables
categorical_columns = list(df4.select_dtypes(include='object'))

# Create new dataframe with categorical variables
df4_cat = df4[categorical_columns]

# Exclude categorical columns from df4
df4 = df4.drop(categorical_columns, axis=1)

df4.shape

(66871, 126)

In [239]:
vif = df4.drop(['play_type'], axis=1)
vif_df = pd.DataFrame()
vif_df['variable'] = vif.columns
vif_df['vif'] = [variance_inflation_factor(vif.values, i) for i in range(vif.shape[1])]
vif_df['vif'] = round(vif_df['vif'], 3)

# Convert inf to zero
vif_df['vif'] = np.where(vif_df['vif'] == np.inf, 0, vif_df['vif'])
vif_df = vif_df.sort_values(by=['vif'], ascending=False)
vif_df

,variable,vif
1,yardline_100,56.504
17,ep,23.392
118,half_seconds_div_score_diff,17.012
113,remaining_downs,12.391
8,ydstogo,10.347
...,...,...
32,dtg_89to75,0.000
14,defteam_score,0.000
36,dtg_34to21,0.000
35,dtg_44to35,0.000


In [240]:
# Create p_df dataframe and add p_value of each variable on play_type
p_df = pd.DataFrame()
p_df['variable'] = df4.columns
p_df['p_value'] = [sm.OLS(df4['play_type'], sm.add_constant(df4[x])).fit().pvalues[1] for x in df4.columns]
p_df = p_df.sort_values(by=['p_value'], ascending=False)
p_df

,variable,p_value
41,prev2_big_play_pass,0.956419
59,prev4_play_def_penalty,0.946516
67,prev2_play_run_inside,0.924586
36,dtg_44to35,0.541264
60,prev5_play_off_penalty,0.517548
...,...,...
26,ol_to_dl_ratio,0.000000
23,defenders_in_box,0.000000
19,wp,0.000000
18,ep,0.000000


In [241]:
# Combine vif and p_df dataframes
multi_df = vif_df.merge(p_df, how='outer', on=['variable'])
multi_df = multi_df.sort_values(by=['p_value'], ascending=False)

In [242]:
# Set vif threshold
vif_threshold = 10.0
p_value_threshold = 0.10

# Return a list of column names with vif > vif_threshold and p-value > p_value_threshold
dropped_columns = list(multi_df.loc[(multi_df['vif'] > vif_threshold) & (multi_df['p_value'] > p_value_threshold), 'variable'])

# Exclusion list of strings from dropped columns
add_back_columns = ['remaining_downs','ydstogo','remaining_yards_per_down','ep']

# Remove add_back_columns from dropped_columns list
dropped_columns = [x for x in dropped_columns if x not in add_back_columns]

print('Dropped columns:', dropped_columns)

Dropped columns: []


In [243]:
# Return a list of column names with vif > vif_threshold and p-value > p_value_threshold
dropped_columns = list(multi_df.loc[(multi_df['vif'] > vif_threshold) | (multi_df['p_value'] > p_value_threshold), 'variable'])

# Remove add_back_columns from dropped_columns list
dropped_columns = [x for x in dropped_columns if x not in add_back_columns]

print('Dropped columns:', dropped_columns)

Dropped columns: ['prev2_big_play_pass', 'prev4_play_def_penalty', 'prev2_play_run_inside', 'dtg_44to35', 'prev5_play_off_penalty', 'game_temp_prod_game_humidity', 'game_wind_div_game_temp', 'prev4_play_off_penalty', 'game_wind_prod_game_temp', 'prev3_play_off_penalty', 'prev5_play_def_penalty', 'spread_line', 'prev3_negative_pass', 'prev2_effct_play', 'half_seconds_div_score_diff', 'yardline_100']


In [244]:
# Drop columns outside of scope
df4.drop(dropped_columns, axis=1, inplace=True)

In [245]:
# Add df4_cat back to df4
df5 = pd.concat([df4, df4_cat], axis=1)

df5.shape

(66871, 133)

In [246]:
df5.columns.to_list()

['week',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'qtr',
 'goal_to_go',
 'ydstogo',
 'play_type',
 'shotgun',
 'no_huddle',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'posteam_score',
 'defteam_score',
 'no_score_prob',
 'safety_prob',
 'ep',
 'wp',
 'total_line',
 'div_game',
 'defenders_in_box',
 'report_eligible',
 'wr_to_db_ratio',
 'ol_to_dl_ratio',
 'intentional_grounding',
 'play_sequence_game',
 'play_sequence_series',
 'score_differential_norm',
 'dtg_99to96',
 'dtg_95to90',
 'dtg_89to75',
 'dtg_74to55',
 'dtg_54to45',
 'dtg_34to21',
 'dtg_20to10',
 'dtg_09to00',
 'prev1_big_play_pass',
 'prev3_big_play_pass',
 'prev1_big_play_run',
 'prev2_big_play_run',
 'prev3_big_play_run',
 'prev1_negative_pass',
 'prev2_negative_pass',
 'prev1_negative_run',
 'prev2_negative_run',
 'prev3_negative_run',
 'prev1_play_off_penalty',
 'prev1_play_def_penalty',
 'prev2_play_off_penalty',
 'prev2_play_def_penalty',
 

## RUN PASS CLASSIFICATION DATAFRAME

In [247]:
df6 = df5.copy()

# Drop column not used in classifier
df6.drop(['intentional_grounding'], axis=1, inplace=True)

# Export binary classifier dataframe
df6.to_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_run_pass_classification_data.csv', index=None, header=True)

df6.shape

(66871, 132)